In [1]:
import pandas as pd
import csv
import time
import datetime
import numpy as np
from datetime import datetime
from time import gmtime
from time import strftime

In [2]:
calendar_dates = pd.read_csv('~/Downloads/google_transit_dublinbus-2/calendar_dates.txt')

In [3]:
calendar_dates.head(10)

,service_id,date,exception_type
0,y1002,20200803,2
1,y1003,20200720,2
2,y1003,20200727,2
3,y1003,20200810,2


In [4]:
calendar = pd.read_csv('~/Downloads/google_transit_dublinbus-2/calendar.txt')

In [5]:
calendar.head(8)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,0,0,1,1,1,0,0,20200715,20200718
1,3,0,0,0,0,0,1,0,20200715,20200718
2,y1001,0,0,0,0,0,1,0,20200719,20200815
3,y1002,1,1,1,1,1,0,0,20200719,20200815
4,y1003,1,0,0,0,0,0,1,20200719,20200815


In [6]:
routes = pd.read_csv('~/Downloads/google_transit_dublinbus-2/routes.txt')

In [7]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,60-1-b12-1,978,1,NaN,3
1,60-102-b12-1,978,102,NaN,3
2,60-104-b12-1,978,104,NaN,3
3,60-11-b12-1,978,11,NaN,3
4,60-111-b12-1,978,111,NaN,3


In [8]:
stop_times = pd.read_csv('~/Downloads/google_transit_dublinbus-2/stop_times.txt')

In [9]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,1.1.60-39A-b12-1.205.I,07:55:00,07:55:00,8240DB007158,1,UCD,0,0,0.00
1,1.1.60-39A-b12-1.205.I,07:55:43,07:55:43,8240DB007048,2,UCD,0,0,750698.41
2,1.1.60-39A-b12-1.205.I,07:56:27,07:56:27,8240DB007699,3,UCD,0,0,1501239.38
3,1.1.60-39A-b12-1.205.I,07:56:54,07:56:54,8240DB007159,4,UCD,0,0,2251660.48
4,1.1.60-39A-b12-1.205.I,07:57:13,07:57:13,8240DB007388,5,UCD,0,0,3001993.17


In [10]:
trips = pd.read_csv('~/Downloads/google_transit_dublinbus-2/trips.txt')

In [11]:
trips.head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,60-11-b12-1,1,3271.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1
1,60-11-b12-1,1,3272.1.60-11-b12-1.15.O,60-11-b12-1.15.O,Saint Pappin's Road - Blackthorn Drive (Blackt...,0
2,60-11-b12-1,1,3273.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1
3,60-11-b12-1,1,3274.1.60-11-b12-1.15.O,60-11-b12-1.15.O,Saint Pappin's Road - Blackthorn Drive (Blackt...,0
4,60-11-b12-1,1,3275.1.60-11-b12-1.16.I,60-11-b12-1.16.I,Blackthorn Road - Saint Pappin's Road,1


#### To verify a route:

- convert route number in routes - ie does it still exist
- if yes... 
- find out the service id from calendars and calendar dates
- use them to get trip_ids in the required direction
- use the trip_ids to get a list of stop numbers
- import the main subroute
- change the stop numbers
- compare

##### get route id

In [12]:
def get_route_id(route):
    route_ids = []
    for index, row in routes['route_short_name'].items():
        if route.lower() == row:
            route_ids.append(routes['route_id'].iloc[index])
    return route_ids

##### get section_id from day of the week and date integer

In [13]:
def get_weekday(date):
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    day_of_week =  datetime.date(date_time_obj).weekday()
    return day_of_week

In [14]:
def get_name_day_of_week(day_of_week):
    days = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
    name_day_of_week = days[day_of_week]
    return name_day_of_week

In [15]:
def get_integer_date(date):
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    timestampStr = date_time_obj.strftime("%Y%m%d")
    date_int = int(timestampStr)
    return date_int

In [16]:
def is_exception(num):
    for index, row in calendar_dates['date'].items():
        if row == num:
            return calendar_dates['service_id'].iloc[index]
    
    return 0

this value is excluded

In [17]:
def get_service_id(date, name_day_of_week, exception_code):
    #poss_rows = []
    if exception_code == 0:
        for index, row in calendar['start_date'].items():
            if date >= row:
                if date <= calendar['end_date'].iloc[index]:
                    if calendar[name_day_of_week.lower()].iloc[index] == 1:
                        service_id = calendar['service_id'].iloc[index]
    else: print("need to fix this")
    return service_id

##### get trip_ids

In [18]:
def get_trip_ids(service_id, route_id, direction):
    route_ids_dataframe = (trips[trips['route_id'].isin(route_id)])
    #print(route_ids_dataframe.head())
    service_id_dataframe = route_ids_dataframe[route_ids_dataframe['service_id'] == service_id]
    direction_id_dataframe = service_id_dataframe[service_id_dataframe['direction_id'] == int(direction)-1]
    trip_ids = list(direction_id_dataframe['trip_id'])
    return trip_ids

##### get list of stops on these trip_ids

In [19]:
def get_list_of_stops(trip_id):
    
    stop_time_dataframe = stop_times[stop_times['trip_id'] == trip_id]
    stops = list(stop_time_dataframe['stop_id'])
    return stops

In [20]:
def how_many_variants(trip_ids):
    unique_patterns = []
    for i in range(0, len(trip_ids)-1):
        if i == 0:
            unique_patterns.append(trip_ids[i])
        if trip_ids[i][-5:] != trip_ids[i+1][-5:]:
            unique_patterns.append(trip_ids[i+1])
    return unique_patterns

In [21]:
def stop_formatter(stop):
    if len(str(stop)) == 0 or len(str(stop)) > 4:
        return "Error"
    elif len(str(stop)) == 1:
        stop_id = '8220DB00000' + str(stop)
    elif len(str(stop)) == 2:
        stop_id = '8220DB0000' + str(stop)
    elif len(str(stop)) == 3:
        stop_id = '8220DB000' + str(stop)
    elif len(str(stop)) == 4:
        stop_id = '8220DB00' + str(stop)
    return stop_id
    

In [90]:
def reverse_stop_formatter(stop):
    if stop[4:6] == 'DB':
        if len(stop) == 12:
            if stop[-6:-1] == '00000':
                stop_id = stop[-1]
            elif stop[-6:-2] == '0000':
                stop_id = stop[-2:]
            elif stop[-6:-3] == '000':
                stop_id = stop[-3:]
            elif stop[-6:-4] == '00':
                stop_id = stop[-4:]
        elif len(stop) == 11:

            if stop[-5:-1] == '0000':
                stop_id = stop[-1]
            elif stop[-5:-2] == '000':
                stop_id = stop[-2:]
            elif stop[-5:-3] == '00':
                stop_id = stop[-3:]
            elif stop[-5:-4] == '0':
                stop_id = stop[-4:]
        
    if stop[4:7] == "VIR":
        if len(stop) == 12:
            if stop[-5:-1] == '0000':
                stop_id = stop[-1]
            elif stop[-5:-2] == '000':
                stop_id = stop[-2:]
            elif stop[-5:-3] == '00':
                stop_id = stop[-3:]
            elif stop[-5:-4] == '0':
                stop_id = stop[-4:]
        
    return stop_id

In [91]:
def report(old_route, direction, old_stops, new_stops):
    #print(old_route, direction)
    #print(old_stops)
    #print(new_stops)
    extensions_new = []
    diversions_new = []
    curtailment_old = [] 
    rerouting_old = []
    if old_stops == new_stops:
        print('No change to route')
        
    elif len(list(np.setdiff1d(new_stops, old_stops))) > 0:
        present_in_new_not_old = list(np.setdiff1d(new_stops, old_stops))
        for stop in present_in_new_not_old:
            print ("Stops presnent in new data that are not in historic data")
            index = present_in_new_not_old.index(stop)
            if index == 0 or index == len(new_stops):
                print("addition at one end of route", stop)
                extensions_new.append(stop)
            else: 
                print("change to middle of route", stop)
                diversions_new.append(stop)
    elif len(list(np.setdiff1d(old_stops, new_stops))) > 0:
        present_in_old_not_new = list(np.setdiff1d(old_stops, new_stops))
        for stop in present_in_old_not_new:
            print ("Stops present in historic data that are not currently served")
            index = old_stops.index(stop)
            if index == 0 or index == len(old_stops):
                print("addition at one end of route", stop)
                curtailment_old.append(stop)
            else: 
                print("change to middle of route", stop)
                rerouting_old.append(stop)
        

In [92]:
dictionary = {'41X': {'1': {'41X_121': [4953, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 909, 786, 790, 7576, 4529, 7585, 7587, 345, 297, 407, 2498, 2499, 2500, 2501, 7623, 5030, 6312, 5031, 6033, 3089, 3090, 3706, 3707, 3708, 3709, 3710, 4925, 3679, 5075, 5076, 3682, 3864, 3865, 4910, 4911, 4912, 4913, 4914, 4915, 4957]}, '2': {'41X_125': [3088, 3704, 3705, 733, 6015, 5029, 5028, 7115, 6310, 6311, 7404, 7405, 7544, 4495, 2809, 2810, 842, 845, 846, 847, 848, 2795, 756, 757, 758, 759, 760, 762, 763, 4953]}}, '25B': {'1': {'25B_274': [7391, 493, 494, 495, 400, 7588, 7392, 312, 1444, 1445, 4407, 2637, 2721, 7012, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 4601, 5058, 4602, 4603, 4604, 4605, 4562, 7386, 4607, 7140, 7141, 7138, 3370, 7231, 7128, 7117, 7118, 6289]}, '2': {'25B_271': [6290, 7120, 7119, 7127, 7143, 3368, 7137, 7139, 4623, 4619, 4561, 4629, 4630, 4631, 4632, 5164, 4633, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 7626, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 347, 406, 4530, 748, 7390]}}, '116': {'1': {'116_1': [848, 2795, 756, 757, 758, 759, 760, 761, 762, 763, 2007, 2008, 2009, 2010, 435, 2011, 2012, 3322, 3563, 444, 445, 4779, 7057, 5142, 448, 4457, 3182, 3183, 4350, 5143, 2837, 2838, 2839, 2840, 2845, 2846, 2847, 4397, 2848, 2849, 2850, 2851, 2852, 4987, 2978, 2979, 2980, 2981, 2954, 2955]}, '2': {'116_3': [2955, 7067, 2956, 2965, 2967, 2968, 2969, 2970, 4988, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2829, 2830, 2831, 2832, 4349, 3184, 449, 450, 3181, 451, 447, 5045, 4460, 453, 4477, 3321, 4727, 4728, 461, 2068, 2069, 2070, 2084, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 907, 908, 909, 786, 792, 4529, 7586, 7587, 7588, 345, 281, 320, 261]}}, '31B': {'1': {'31B_44': [621, 289, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 525, 526, 527, 528, 529, 530, 531, 709, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 688, 689, 690, 5062, 691, 692, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707]}, '2': {'31B_46': [557, 558, 559, 560, 562, 563, 564, 565, 566, 707, 708, 731, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 680, 681, 682, 683, 684, 685, 686, 688, 689, 690, 5062, 691, 692, 693, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 7569]}}, '68': {'1': {'68_81': [4495, 7546, 7547, 1278, 1282, 4456, 1284, 1285, 1286, 1288, 1289, 1360, 1362, 1363, 1364, 1365, 1366, 1367, 1369, 1370, 1371, 1372, 1373, 1374, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 4379, 1968, 1969, 1970, 1971, 2114, 2115, 6011, 2133, 6244, 2134, 3378, 3379, 4669, 2165, 2166, 2167, 3383, 3384, 3385, 3386, 3387, 4559, 3389, 3390, 6119, 3392, 3393, 3394, 3396, 3397, 3398, 3399, 6187, 6188, 6215, 6184]}, '2': {'68_83': [6185, 6186, 6216, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3410, 3407, 3408, 3409, 4558, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3422, 3423, 2156, 6014, 2158, 2159, 4409, 1972, 1973, 1974, 1975, 1977, 1978, 1980, 1981, 1982, 4406, 1984, 1985, 1986, 1987, 1988, 1989, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1358, 7665]}}, '122': {'1': {'122_18': [4525, 4526, 1700, 1701, 1702, 16, 73, 74, 4371, 832, 833, 77, 78, 79, 80, 82, 83, 84, 51, 52, 263, 271, 1278, 1282, 4456, 1284, 1285, 1286, 1288, 1289, 1360, 1362, 1363, 1364, 1365, 1366, 1367, 7455, 7438, 1410, 1411, 1412, 1413, 1414, 1415, 1417, 1418, 5149, 1419, 6094]}, '2': {'122_20': [1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1432, 1433, 1434, 1435, 7439, 7454, 1380, 1381, 1382, 1383, 1384, 1385, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1359, 279, 3, 12, 14, 53, 54, 55, 56, 58, 59, 60, 820, 821, 63, 64, 65, 66, 4498, 1660, 1661, 4523, 4524, 4322]}}, '66A': {'1': {'66A_37': [7391, 493, 494, 495, 400, 346, 317, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 3365, 3366, 3367, 3886, 3887, 3888, 7186, 7187, 3954, 3955, 3956, 3957, 3993, 3994]}, '2': {'66A_38': [3994, 3995, 3996, 3997, 3998, 3999, 4000, 4001, 3949, 3950, 3372, 3373, 3374, 2233, 2232, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 2810, 7387]}}, '15B': {'1': {'15B_60': [395, 396, 397, 398, 399, 400, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 2914, 2915, 2916, 2917, 2918, 1300, 1301, 1302, 1303, 5129, 5130, 1135, 7064, 7066, 2924, 2925, 7442, 7446, 7070, 7445, 6339, 6324, 6325, 6341, 6277, 6278, 6279, 6280]}, '2': {'15B_61': [6285, 6286, 6287, 6288, 6342, 6298, 6336, 7069, 7443, 7444, 2941, 2942, 7065, 7063, 1137, 5131, 5132, 1331, 1332, 1333, 1334, 2985, 2986, 2988, 3001, 3002, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 340, 407, 2498, 2499, 7216, 7558, 353, 354]}}, '76A': {'1': {'76A_28': [7026, 4747, 2960, 1840, 1841, 1842, 7374, 2241, 2242, 2243, 2200, 5006, 5007, 2655, 2656, 2668, 2672, 2673, 2206, 4798, 4795, 2675, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2158, 2159, 4409, 2173, 2174, 3425, 2763, 2619, 2620, 2345, 5008, 4640, 4347, 4341]}, '2': {'76A_29': [4341, 4348, 4646, 4647, 2363, 2364, 2618, 5011, 2762, 2110, 2111, 2112, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2123, 2685, 4795, 2686, 7510, 4799, 2205, 2687, 2688, 2689, 2696, 2697, 4414, 5009, 2201, 4401, 1813, 1814, 7026]}}, '43': {'1': {'43_84': [1184, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 1199, 1200, 4385, 1201, 1202, 1203, 6115, 4849, 1205, 1206, 1207, 1208, 1209, 1210, 3575, 3576, 3648, 3649, 3650, 7328, 3651, 3652, 3653, 3654, 5084, 6054, 4330, 3679, 3711, 3712, 6359, 7189]}, '2': {'43_88': [5044, 4433, 3752, 3751, 3689, 7594, 3690, 6117, 3655, 5086, 3656, 3657, 3658, 7329, 3659, 3660, 3647, 3661, 3573, 3662, 1212, 1213, 1214, 1215, 1216, 1217, 4563, 1218, 1270, 1272, 1273, 1274, 1275, 1276, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 1184]}}, '67X': {'1': {'67X_43': [4952, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 6058, 791, 7576, 4529, 7585, 7587, 345, 318, 312, 1444, 1445, 1447, 2637, 2721, 7012, 2201, 4401, 2213, 2214, 2215, 7184, 7050, 7185, 7186, 7187, 3947, 3892, 3893, 3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3904, 3905, 7085, 7086, 7087, 7088, 7089, 7091, 3910, 3911, 3912, 3913, 3914, 3925, 3008, 6089]}, '2': {'67X_46': [3921, 3922, 3923, 7079, 7080, 7081, 7565, 7082, 7083, 7084, 7108, 3935, 3936, 3938, 3939, 3940, 3941, 3943, 3944, 3945, 3946, 4001, 7193, 7049, 2236, 2239, 2241, 2242, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 4495, 2809, 2810, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 4952]}}, '150': {'1': {'150_8': [4495, 1279, 1934, 2310, 2311, 5025, 4854, 1438, 4855, 1439, 1440, 1386, 1387, 1388, 1389, 3355, 3356, 1441, 1442, 7414, 1396, 2317, 2318, 2319, 2320, 2321, 2322, 6163, 1095, 1096, 1097, 1099, 6132, 4861, 4862]}, '2': {'150_9': [4863, 4864, 4865, 4577, 4578, 1100, 100, 1111, 2323, 2324, 2325, 2326, 5150, 2327, 2328, 2329, 2330, 2331, 1397, 1398, 1399, 5148, 1401, 1402, 1403, 1404, 1405, 4856, 4857, 307, 308, 309, 7412, 7413, 2383, 2003, 1358, 4494]}}, '38B': {'1': {'38B_40': [7333, 783, 784, 785, 786, 793, 7576, 401, 274, 2, 192, 795, 796, 57, 58, 59, 60, 1802, 1795, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 1816, 6171, 4974, 4976, 7126, 7124, 7343, 7318, 4980, 2289, 1822, 4994, 4995, 4996, 1826, 1827, 7382, 7381, 4742, 4743, 4745]}, '2': {'38B_44': [4745, 4989, 4990, 4991, 4992, 1719, 1828, 1829, 4997, 4998, 4999, 1833, 1834, 1835, 1836, 1837, 2338, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1804, 1803, 77, 78, 79, 80, 82, 817, 818, 819, 265, 6059, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7370, 7339]}}, '14C': {'1': {'14C_17': [248, 249, 250, 251, 252, 255, 234, 235, 236, 1190, 1191, 1192, 1193, 1194, 1195, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 4415, 301, 336]}, '2': {'14C_18': [6041, 2826, 2842, 2843, 2844, 2845, 2846, 2847, 4397, 2848, 2849, 2850, 2851, 2852, 4707, 4708, 4336, 1051, 1052, 1053, 1054, 1036, 4709, 1037, 1038, 2998, 2999, 3000, 3001, 3002, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 297]}}, '37': {'1': {'37_14': [7340, 783, 784, 785, 786, 793, 7576, 4529, 7586, 7587, 7588, 329, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 802, 1654, 1655, 1656, 1657, 4458, 1658, 1659, 1660, 1661, 1662, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 7380, 1676, 1517, 1678, 1679, 4520, 4825, 7233, 4573, 4893, 4894, 4895, 7217, 7227, 6172, 4493, 4362, 2959]}, '2': {'37_15': [4747, 1882, 7379, 4903, 7219, 7218, 7031, 4897, 4898, 4574, 4575, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 1528, 1710, 1711, 1712, 1713, 1714, 1715, 7453, 1478, 1479, 315, 313, 349, 404, 747, 843, 845, 846, 7340]}}, '79': {'1': {'79_10': [326, 1443, 1444, 1445, 4407, 2637, 2638, 2640, 2641, 2643, 2644, 2645, 2646, 2647, 2648, 2649, 2650, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2909, 2906, 2907, 2908]}, '2': {'79_11': [2670, 2910, 2911, 7507, 2663, 2664, 2665, 2666, 2667, 2696, 2697, 2700, 2701, 2702, 2703, 2704, 2706, 6070, 2708, 5134, 2718, 2719, 7435, 2722, 4413, 1476, 7453, 1480, 2912, 1358, 326]}}, '77X': {'1': {}, '2': {'77X_61': [4713, 4714, 4960, 4932, 4933, 2590, 2591, 2592, 2357, 2566, 2567, 2568, 2569, 2570, 7222, 6170, 5023, 5024, 5161, 5162, 4880, 2606, 2607, 2608, 2609, 2610, 2554, 2425, 2426, 2427, 2428, 2429, 2430, 2371, 2372, 2373, 1424, 2190, 4434, 2379, 2380, 2382, 5099, 2383, 2002, 1358, 5192, 350, 2809, 2810, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 4952]}}, '69': {'1': {'69_45': [4495, 4720, 1443, 1444, 1445, 7078, 1449, 1450, 1451, 1452, 2640, 2641, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 4379, 1968, 1969, 1970, 1971, 2160, 2161, 4665, 4666, 4667, 2162, 2163, 2168, 4536, 6003, 5123, 5124, 5125, 6001, 4438, 3432, 3426, 3434, 3435, 3453, 3437, 3439, 3440, 4557, 3441, 4560, 5126]}, '2': {'69_47': [4564, 3443, 3444, 3445, 3446, 3447, 3448, 3449, 3436, 7432, 3454, 3455, 3456, 3457, 3438, 5022, 6002, 5119, 5120, 5121, 5122, 3461, 4532, 4534, 4535, 2169, 2170, 4672, 3465, 4673, 2172, 2159, 4409, 1972, 1973, 1974, 1975, 1977, 1978, 1980, 1981, 1982, 4406, 1984, 1985, 1986, 1987, 1988, 2642, 7435, 2722, 1471, 1472, 4408, 1473, 1474, 1476, 7453, 1480, 2912, 1358, 7665]}}, '44B': {'1': {'44B_60': [2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 6006, 3493, 3494, 3495, 3496, 3497, 3498, 3499, 3500, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3510, 3512, 3514]}, '2': {'44B_61': [3514, 3515, 3516, 3517, 3518, 3519, 3521, 3523, 3524, 3525, 3527, 3528, 3529, 3532, 3533, 3491, 6007, 2836, 2837, 2838, 2839, 2840, 2841, 2865, 2866, 2825]}}, '54A': {'1': {'54A_11': [342, 1279, 1934, 2385, 2386, 2387, 2388, 1290, 1291, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2401, 2402, 2403, 1096, 1097, 1099, 6132, 2551, 2553, 2554, 2555, 2556, 2557, 4436, 5008, 4640, 7214, 4347, 2349, 2351, 2558, 6170, 5023, 5024]}, '2': {'54A_12': [5161, 5162, 2571, 2359, 2360, 4348, 4646, 4647, 4435, 2594, 2595, 2596, 2597, 2599, 4577, 4578, 1100, 100, 1111, 2404, 2405, 2406, 2408, 2409, 2410, 2411, 2412, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 1344, 1345, 1347, 2634, 2635, 2636, 2383, 2003, 1358, 342]}}, '239': {'1': {'239_26': [7026, 1882, 7379, 4903, 1883, 1884, 7155, 1887, 1888, 1889, 1890, 4820, 6126, 2226, 6134, 2223, 2224, 2225, 4620, 7563, 3368, 4626, 4627, 4327, 4603, 4604, 2681, 4691, 2682, 4690, 4411, 4715, 4795]}, '2': {'239_28': [4795, 4715, 4688, 4689, 4410, 2678, 4630, 4631, 5138, 4611, 4612, 4613, 3370, 7168, 4616, 2227, 2229, 6133, 2228, 6127, 4826, 1912, 1895, 1896, 1897, 1844, 6073, 4493, 4362, 7026]}}, '1': {'1': {'1_37': [226, 228, 229, 227, 230, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 265, 271, 340, 350, 351, 352, 353, 354, 355, 356, 357, 390, 372, 373, 374, 375, 2804, 376, 377, 378, 380]}, '2': {'1_40': [381, 382, 4451, 383, 384, 385, 387, 388, 389, 393, 371, 391, 392, 395, 396, 397, 398, 399, 400, 319, 278, 7615, 10, 12, 14, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 221, 222, 223, 224, 225, 226]}}, '51D': {'1': {'51D_50': [7392, 1443, 1444, 1445, 4407, 2637, 2638, 2721, 7012, 2201, 4401, 2213, 4688, 4689, 4410, 2678, 2679, 2680, 4677, 2136, 2137, 7201, 2138, 2139, 2140, 2141, 2142, 2143, 2145, 4670, 3420, 3421, 3422, 4671, 4672, 3465, 4673, 2172, 4409, 4550, 4551, 1965, 1966]}, '2': {'51D_51': [4664, 2160, 4665, 4666, 4667, 4668, 3376, 3377, 3378, 3379, 4669, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 6245, 6013, 6247, 2681, 4691, 4796, 4690, 4411, 2239, 2241, 2242, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 4495, 2809, 2810, 2811, 749, 750, 751, 752, 753, 754, 755]}}, '15': {'1': {'15_16': [6318, 6319, 7246, 6320, 4594, 4595, 4596, 4563, 1218, 1270, 1272, 1273, 1274, 1275, 1276, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 4415, 301, 4495, 5190, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1121, 1122, 1123, 1124, 1125, 1127, 1130, 1140, 1141, 1142, 1143, 1144, 1145, 4869, 4870, 3007, 6283, 6282]}, '2': {'15_17': [6282, 6335, 6326, 4886, 4887, 1150, 1151, 1152, 1153, 1154, 946, 1155, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 299, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 1199, 1200, 4385, 1201, 1202, 1203, 6115, 4593, 7516, 6079, 6315, 6316, 7236, 7245, 6317]}}, '31': {'1': {'31_15': [621, 289, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 525, 526, 527, 528, 529, 530, 531, 709, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 557, 558, 559, 560, 562, 563, 564, 565, 566, 567]}, '2': {'31_18': [567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 7569]}}, '102': {'1': {'102_8': [4381, 935, 936, 913, 914, 938, 940, 941, 942, 943, 4503, 3599, 3610, 3611, 6135, 3612, 3613, 3615, 3616, 3618, 3619, 3620, 3622, 3623, 4387, 3634, 3635, 3636, 905, 4331, 3641, 4339, 6010, 3656, 3653, 1039, 1040, 6054, 4330, 3691, 3572, 3701, 3702, 3088, 3704, 3705, 733, 6015, 788, 815, 816, 3663, 7348]}, '2': {'102_9': [7348, 3669, 3671, 3672, 3078, 3087, 6033, 3089, 3090, 3706, 3707, 3708, 3709, 3710, 4925, 4926, 3678, 6117, 1041, 1050, 3656, 3653, 6055, 1073, 3641, 3642, 3643, 3583, 3584, 3585, 3586, 3587, 3588, 3590, 3589, 3591, 3592, 3605, 3606, 3607, 3608, 3609, 3598, 4465, 944, 945, 947, 948, 949, 928, 951, 952, 953, 4381]}}, '33B': {'1': {'33B_56': [4330, 3679, 3711, 3712, 3713, 3714, 3719, 3718, 3720, 3721, 3722, 5069, 3723, 3724, 3725, 3726, 7039, 7204, 3728, 3729, 3730, 3732]}, '2': {'33B_58': [3732, 3733, 3734, 3735, 3736, 7204, 3739, 3738, 7170, 3740, 3741, 5165, 7169, 3743, 3780, 3744, 3745, 3746, 3749, 3750, 3752, 3751, 3689, 7594, 3690, 4330]}}, '41C': {'1': {'41C_78': [288, 1171, 1172, 1173, 1174, 1175, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 3670, 3671, 3672, 3674, 3675, 5030, 5031, 6033, 3089, 3090, 3706, 3707, 3708, 3709, 3710, 4925, 4926, 3678, 3679, 3711, 3712, 4916, 4917, 4918, 4919, 4920, 4910, 4911, 4912, 4913, 4914, 4915, 4957]}, '2': {'41C_79': [4843, 4905, 7655, 4906, 4907, 4908, 4909, 4921, 4922, 4923, 4924, 4958, 3752, 3751, 3689, 3691, 3572, 3701, 3702, 3088, 3704, 3705, 733, 6015, 5029, 5028, 3696, 3697, 3698, 3699, 7593, 3885, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 277, 288]}}, '27A': {'1': {'27A_4': [300, 496, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 4782, 1782, 1783, 1784, 1785, 1786, 624, 625, 626, 627, 628, 629, 630, 4466, 633, 634, 1005, 1788, 1789, 7199]}, '2': {'27A_5': [1791, 1792, 1793, 962, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 1796, 1797, 1798, 1799, 1800, 1801, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 620, 7568]}}, '39A': {'1': {'39A_40': [767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 793, 7576, 4529, 7586, 7587, 7588, 328, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 1913, 1914, 1805, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 7389, 7025, 4464, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1899, 6107, 6108, 6109, 6110, 7020, 7029, 7038, 7011, 2171, 7160, 7047, 7161, 7162]}, '2': {'39A_43': [7158, 7048, 7159, 7388, 7017, 7018, 7030, 7021, 6111, 6112, 6113, 1898, 1858, 1859, 1860, 4492, 1861, 1862, 1863, 1864, 1865, 1866, 1850, 1867, 4489, 4747, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1906, 1907, 1908, 1909, 1713, 1714, 1715, 7453, 1478, 1479, 315, 313, 335, 349, 404, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 767]}}, '40D': {'1': {'40D_102': [6004, 7, 11, 14, 15, 834, 835, 836, 837, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1538, 4548, 1514, 980, 981, 1576, 1577, 1578, 4364, 1579, 1580, 7237, 7241, 7247, 7243, 7126, 7124, 7343, 4978, 4979, 4986, 7107, 7102, 7103, 7104, 7105, 7106, 6005, 7072]}, '2': {'40D_104': [7073, 7097, 7098, 7099, 7100, 7101, 5071, 5072, 7342, 7125, 7344, 2510, 7242, 7238, 7224, 1581, 1582, 4793, 6068, 1584, 983, 984, 985, 1529, 4542, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 186, 838, 839, 840, 841, 49, 51, 52, 4725, 6071]}}, '7D': {'1': {'7D_120': [4962, 4725, 6059, 334, 406, 747, 842, 845, 846, 847, 848, 2795, 756, 757, 758, 759, 760, 761, 762, 763, 2007, 2008, 2009, 2010, 435, 7353, 7584, 3344, 7421, 7422, 6085, 2053, 2054, 2055, 4565, 4566, 4567, 3238, 3239, 3240, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057]}, '2': {'7D_123': [3057, 4830, 7667, 3091, 3092, 3093, 3535, 3536, 3537, 3538, 7556, 3539, 3540, 3541, 3542, 3543, 3141, 3142, 3224, 3225, 3226, 3227, 3228, 3229, 7472, 3230, 3232, 3233, 3234, 4462, 3235, 3236, 3237, 3238, 3239, 3240, 3068, 3069, 3070, 2041, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3084, 469, 470, 471, 472, 473, 4705, 475, 476, 477, 478, 7053, 2088, 2089, 770, 771, 772, 417, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495, 7587, 345, 320, 7492, 281, 4, 4962]}}, '120': {'1': {'120_7': [6004, 3, 192, 795, 796, 56, 58, 4499, 4934, 4500, 4481, 4485, 822, 823, 824, 825, 826, 827, 71, 72, 5144, 7027, 4538, 7396, 5145, 276]}, '2': {'120_9': [284, 4504, 7028, 5147, 68, 30, 69, 828, 829, 830, 831, 75, 4501, 4935, 4502, 4936, 79, 80, 82, 817, 818, 819, 263, 6071]}}, '39': {'1': {'39_20': [7333, 783, 784, 785, 786, 793, 7576, 7586, 7587, 7588, 328, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 1913, 1914, 1805, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 1813, 1814, 1852, 1853, 1854, 1844, 6172, 4493, 4362, 7495, 7025, 4464, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1899, 1887, 1888, 1889, 1890, 7240, 1891, 7156, 7020, 7029, 7038, 7011, 2171, 7160, 7047, 7161, 7162]}, '2': {'39_21': [7158, 7048, 7159, 7388, 7017, 7018, 7030, 7021, 7163, 1893, 1894, 1912, 1895, 1896, 1897, 6088, 1898, 1858, 1859, 1860, 4492, 1861, 1862, 1863, 1864, 1865, 1866, 1850, 1867, 4489, 4747, 1882, 7379, 4903, 1883, 1884, 1902, 1903, 1904, 1840, 1841, 1842, 7374, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1906, 1907, 1908, 1909, 1713, 1714, 1715, 7453, 1478, 1479, 315, 313, 335, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7339]}}, '70': {'1': {'70_60': [7333, 783, 784, 785, 786, 793, 7576, 7586, 7587, 7588, 329, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 1913, 1914, 1805, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 7389, 4324, 4769, 4770, 4765, 4766, 4767, 4768, 4325, 3328, 3329, 3330, 3331, 3351, 3332, 3333]}, '2': {'70_67': [3333, 3334, 3352, 3335, 3336, 3337, 3338, 3339, 3340, 4324, 4769, 4770, 4765, 4766, 4767, 4768, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1906, 1907, 1908, 1909, 1713, 1714, 1715, 7453, 1478, 1479, 315, 313, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7370, 7339]}}, '111': {'1': {'111_7': [7639, 7640, 5047, 3134, 3143, 3144, 3221, 3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 7472, 3230, 3232, 3233, 3234, 4462, 3235, 3236, 3237, 6103, 7554, 2031, 2032, 2033, 2034, 2035, 2036, 2039, 2037, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057]}, '2': {'111_8': [3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 4983, 7647, 2042, 2043, 2044, 2045, 2046, 7553, 6082, 3205, 3206, 3207, 3208, 3209, 3210, 4981, 3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3134, 3143, 3144, 5046, 7639]}}, '41D': {'1': {'41D_26': [288, 1171, 1172, 1173, 1174, 1175, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 3670, 3671, 3672, 3674, 3675, 3676, 5073, 5074, 6054, 4330, 3679, 3711, 7189]}, '2': {'41D_27': [5044, 3751, 3689, 7594, 3690, 6117, 5079, 3694, 3695, 3696, 3697, 3698, 3699, 3885, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 277, 288]}}, '47': {'1': {'47_139': [7662, 7564, 340, 350, 351, 352, 353, 354, 355, 356, 357, 390, 372, 373, 374, 375, 2804, 376, 377, 378, 380, 2087, 7053, 2088, 2089, 763, 2007, 2008, 2009, 2010, 435, 2011, 2012, 3322, 3563, 444, 445, 4779, 7057, 5142, 7669, 3187, 7567, 7326, 7418, 7417, 3471, 3472, 3473, 5015, 7290, 5016]}, '2': {'47_136': [5013, 5014, 7291, 3485, 3486, 3487, 7415, 7416, 7325, 4984, 448, 3181, 451, 447, 5045, 4460, 453, 4477, 3321, 2066, 2067, 4728, 461, 2068, 2069, 2070, 2084, 768, 2085, 2086, 4455, 381, 382, 4451, 383, 384, 385, 7529, 387, 388, 389, 393, 371, 391, 392, 395, 396, 397, 398, 399, 400, 7564, 7663]}}, '27X': {'1': {'27X_43': [4952, 768, 2085, 2086, 479, 480, 481, 482, 483, 485, 488, 491, 493, 494, 495, 496, 7490, 7659, 497, 523, 4382, 1196, 1197, 1198, 1199, 1200, 4385, 1201, 4695, 1243, 4696, 4697, 1250, 4698, 4383, 4699, 4388, 4700, 1266, 1267, 1268, 1269, 1265, 6115, 4593, 4594]}, '2': {'27X_42': [4595, 4596, 4563, 1218, 1258, 1259, 1260, 6130, 1262, 4692, 1245, 4693, 1254, 1255, 1256, 4694, 1249, 1247, 1248, 1273, 1274, 1275, 1276, 1277, 1219, 1220, 1221, 664, 614, 4415, 302, 4495, 2809, 405, 408, 409, 411, 414, 416, 417, 418, 419, 420, 2088, 2089, 763, 4952]}}, '33E': {'1': {'33E_74': [292, 1171, 1172, 1173, 1174, 1175, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 3670, 3671, 3672, 3674, 3675, 3676, 3677, 3678, 3679, 3711, 3712, 3713, 3714, 3719, 3718, 3720, 7691, 3721, 3722, 5069, 3723, 3724, 3725, 3738, 7170, 3740, 3741, 5165, 7169, 3743, 3780, 3744, 3755, 3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765, 3766, 3767, 3768, 3770, 3769, 3771, 3772, 3773, 5176, 3774, 7172, 3775, 3777, 3778, 7690, 3781, 3782, 3783, 3784, 3785, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3797, 3798, 6076, 6077, 3569, 3571, 3801, 3802]}, '2': {}}, '236': {'1': {'236_9': [4747, 2960, 1545, 1816, 6171, 4974, 4976, 7126, 7124, 7343, 7575, 4978, 4979, 4986, 7107, 7102, 7103, 7104, 7105, 7106, 7381, 4742, 4743, 4744, 4745]}, '2': {'236_10': [4745, 4989, 4990, 4991, 4992, 4986, 7107, 7102, 7103, 7104, 7105, 7106, 5071, 5072, 4321, 7570, 7342, 7125, 7344, 7334, 6274, 1838, 2338, 661, 7026]}}, '39X': {'1': {'39X_1': [7333, 783, 785, 786, 793, 7586, 7587, 328, 1443, 1647, 1649, 1911, 1805, 1662, 1666, 1808, 7475, 6107, 6109, 6110, 7029, 7038, 7011, 7160, 7047, 7161, 7162]}, '2': {'39X_3': [7158, 7048, 7159, 7017, 7018, 7030, 6111, 6112, 6113, 4323, 1845, 1696, 1700, 1906, 1909, 1713, 1715, 1478, 315, 4495, 2809, 2810, 749, 751, 752, 7339]}}, '17': {'1': {'17_10': [1380, 1381, 1406, 1407, 1409, 3353, 3354, 3355, 3356, 1400, 1391, 3357, 3358, 2449, 2450, 2451, 2434, 2435, 2436, 2437, 1117, 1118, 1119, 1120, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1055, 2868, 2892, 2893, 2894, 2895, 3012, 861, 862, 863, 864, 865, 866, 2052, 2082, 765, 2007, 2083, 2008, 2009, 2010, 462, 464, 465, 466, 467, 468, 3032, 3033, 3085]}, '2': {'17_15': [3085, 3082, 3083, 430, 431, 432, 433, 434, 2068, 2069, 2070, 2084, 4391, 765, 867, 2071, 869, 2072, 870, 873, 874, 875, 876, 3011, 2820, 2822, 2823, 5032, 2867, 1035, 1314, 1316, 1315, 1317, 1318, 1319, 1281, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1086, 1087, 1088, 1089, 1090, 2413, 2414, 2415, 2327, 2464, 2465, 2466, 3360, 1392, 1394, 1395, 3361, 3362, 1436, 3952, 2190, 1366, 1367]}}, '33': {'1': {'33_44': [292, 1171, 1172, 1173, 1174, 1175, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 3670, 3671, 3672, 3674, 3675, 3676, 3677, 3678, 3679, 3711, 3712, 3713, 3714, 3715, 5175, 3755, 3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765, 3766, 3767, 3768, 3770, 3769, 3771, 3772, 3773, 3774, 7172, 3775, 3777, 3778, 3781, 7690, 3782, 3783, 3784, 3785, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3797, 3798, 3799, 6076, 6077, 3569, 3571, 3801, 3802, 3805, 7175, 3806, 3807, 3808, 3809, 7174]}, '2': {'33_70': [6048, 5064, 7173, 3811, 3812, 3664, 3813, 3814, 7592, 3815, 3816, 3566, 7205, 3567, 7292, 3818, 3819, 3820, 3822, 3823, 3824, 3565, 3825, 3826, 3827, 3828, 3829, 3831, 3832, 3833, 3834, 3835, 3836, 3837, 3838, 3840, 3842, 3843, 3844, 3845, 3846, 3848, 3849, 3850, 3851, 3852, 3853, 3854, 3855, 3856, 3857, 3858, 3859, 3860, 3861, 3862, 3863, 3748, 6050, 3749, 3750, 3752, 3751, 3689, 7594, 3691, 3692, 3694, 3695, 3696, 3697, 3698, 3699, 3885, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 277, 292]}}, '142': {'1': {'142_13': [3605, 3606, 3607, 6051, 6053, 7327, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 905, 4331, 4339, 6010, 7692, 3656, 7194, 7196, 6310, 6311, 7399, 7398, 7397, 6252, 301, 4495, 2809, 2810, 843, 1013, 1015, 1016, 1017, 1018, 1019, 1020, 1076, 1021, 6042, 1022, 1023, 1024, 895, 1025, 1026, 1027, 1028, 2818, 2819, 3012, 877]}, '2': {'142_12': [877, 876, 3011, 2896, 2897, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 407, 2498, 2499, 2500, 2501, 7623, 6312, 7197, 7195, 3653, 6055, 1073, 3641, 3642, 3643, 3583, 3584, 3585, 3625, 3626, 3627, 3628, 6052, 3613, 3615, 3616]}}, '65B': {'1': {'65B_65': [7564, 4521, 1283, 4456, 1284, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1121, 1122, 1123, 1124, 1125, 1127, 1130, 2550, 2516, 2517, 2518, 4749, 4750, 3006, 4871, 4872, 4873, 4874, 2532, 4877, 4878, 7180, 7331, 2559, 2560, 2561, 2562, 2564, 4927, 4928, 2352, 2573, 2574, 2575, 4929, 7460, 4930, 4931, 7459]}, '2': {'65B_66': [4713, 4714, 4960, 4932, 4933, 2590, 2591, 2592, 2357, 2566, 2567, 2568, 2569, 7550, 2570, 7222, 6170, 7183, 4880, 2539, 2540, 4882, 4883, 4884, 4885, 6343, 4759, 4760, 2547, 2548, 2600, 1155, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 1352, 1353, 7589, 1355, 1357, 1358, 7564]}}, '77A': {'1': {'77A_29': [395, 396, 397, 398, 399, 400, 4522, 1934, 2310, 2311, 2312, 2313, 2314, 2315, 2094, 1406, 1407, 1409, 2095, 2096, 2097, 2099, 2101, 2102, 2103, 2332, 2333, 2334, 4662, 2335, 2336, 2337, 2339, 2420, 2421, 2423, 2424, 2447, 5133, 2596, 2611, 2602, 2613, 2614, 2615, 2535, 2536, 2616, 2617, 2557, 4436, 5008, 4640, 4347, 2349, 2351, 2558, 2559, 2560, 2561, 2562, 2564, 4927, 4928, 2352, 2573, 2574, 2575, 4929, 7460, 4930, 4931, 7459]}, '2': {'77A_28': [4713, 4714, 4960, 4932, 4933, 2590, 2591, 2592, 2357, 2566, 2568, 2567, 2569, 7550, 2570, 2571, 2359, 2360, 4348, 4646, 4647, 2346, 4435, 2604, 2605, 2538, 2606, 2607, 2608, 2609, 2610, 2554, 2425, 2426, 2427, 2428, 2429, 2430, 2371, 2372, 2373, 2377, 2378, 2183, 2184, 2185, 7043, 1424, 2186, 2187, 2188, 2189, 1436, 3952, 2190, 4434, 2379, 2380, 2382, 5099, 2383, 2002, 1358, 340, 350, 351, 352, 353, 354]}}, '161': {'1': {'161_50': [2825, 2867, 1035, 1314, 1316, 1315, 4332, 4333, 4334, 4712, 4582, 4987, 2979, 2980, 2981, 7447, 2954, 7067, 2956, 2965, 2983, 2984, 7449, 2927, 2929, 2930, 7441, 2933]}, '2': {'161_51': [2934, 2935, 2936, 2937, 2938, 7440, 2940, 2948, 2949, 2950, 2954, 7067, 2956, 2965, 7448, 2967, 2968, 2969, 2970, 4988, 4707, 4708, 4336, 4337, 1310, 1311, 1312, 1313, 1055, 2868, 2825]}}, '44': {'1': {'44_40': [7571, 207, 208, 209, 210, 211, 212, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 265, 271, 340, 350, 2809, 2810, 2811, 843, 1013, 1015, 2812, 2813, 2814, 852, 853, 854, 855, 2815, 2816, 2817, 2818, 2819, 2820, 2822, 2823, 5032, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 7375, 3466, 3467, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3280, 3282, 3479, 4086, 4087, 4088, 4089, 4090, 4091, 6097, 4093, 4094, 4096]}, '2': {'44_36': [4096, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 3283, 3284, 3310, 3481, 3482, 3483, 3484, 3485, 3486, 3487, 3488, 3489, 3492, 2835, 7176, 2836, 2837, 2838, 2839, 2840, 2841, 2865, 2866, 2892, 2893, 2894, 2895, 2896, 2897, 2898, 3562, 2899, 884, 885, 886, 887, 2900, 2901, 2902, 2903, 1074, 6074, 5034, 793, 4529, 495, 400, 319, 278, 7615, 10, 12, 14, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 206, 207, 208, 209, 210, 1643, 7571]}}, '270': {'1': {'270_42': [7026, 4323, 4324, 4769, 4770, 4765, 4766, 4767, 4768, 4325, 3328, 3329, 3330, 3331, 3351, 3332, 3333]}, '2': {'270_44': [3333, 3334, 3352, 3335, 3336, 3337, 3338, 3339, 3340, 4324, 4769, 4770, 4765, 4766, 4767, 4768, 4326, 7026]}}, '66X': {'1': {'66X_92': [4952, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 791, 6058, 7576, 7585, 4529, 7587, 345, 318, 312, 1444, 1445, 1447, 2637, 2721, 7012, 2201, 4401, 2213, 2214, 2215, 7184, 7050, 7185, 7186, 7187, 3954, 3955, 3956, 3957, 3958, 3959, 3960, 6156, 6165, 6157, 6158, 7314, 3964, 3965, 3968, 3966, 5113, 3918, 5114]}, '2': {'66X_102': [3980, 6089, 3981, 3982, 3983, 3985, 3986, 3987, 3989, 3990, 3991, 3998, 3999, 4000, 4001, 7193, 7049, 2236, 2239, 2241, 2242, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 4495, 2809, 2810, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 4952]}}, '26': {'1': {'26_28': [7391, 493, 494, 495, 400, 346, 317, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2202, 2203, 2210]}, '2': {'26_29': [2210, 2211, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 2810, 7387]}}, '4': {'1': {'4_10': [324, 327, 7478, 7113, 7479, 127, 7480, 112, 7481, 113, 114, 115, 37, 38, 39, 40, 146, 147, 184, 185, 186, 187, 188, 189, 190, 191, 819, 7457, 4725, 273, 405, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 6334, 7660, 3114, 3115, 3116, 6085, 2026, 7226]}, '2': {'4_12': [7330, 2051, 6086, 3161, 3162, 3163, 3164, 3084, 469, 470, 471, 472, 473, 4705, 475, 476, 477, 478, 479, 480, 481, 482, 483, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495, 400, 281, 6, 194, 195, 196, 197, 198, 199, 200, 201, 148, 149, 150, 27, 28, 29, 4680, 90, 91, 92, 93, 94, 126, 6182, 322, 323, 332]}}, '83': {'1': {'83_16': [324, 7500, 1555, 1556, 1557, 1181, 7350, 6230, 6228, 4788, 6273, 4891, 7458, 7425, 7424, 7423, 1566, 1567, 4372, 179, 7431, 175, 176, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 1614, 1615, 1478, 1479, 315, 336, 1279, 1283, 4456, 1284, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2448, 5026, 2450, 2490, 2491, 2492]}, '2': {'83_22': [2492, 2493, 2465, 2466, 2467, 2494, 2495, 2496, 2497, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 1170, 1069, 1070, 1071, 4528, 1072, 1352, 1353, 1355, 1357, 1359, 319, 1443, 1616, 1617, 1618, 1619, 195, 196, 197, 198, 199, 200, 201, 202, 151, 153, 154, 155, 156, 7430, 177, 178, 1539, 1540, 7426, 7427, 7428, 138, 133, 134, 4890, 4369, 6219, 6229, 6348, 1182, 1550, 1551, 1552, 6234, 332]}}, '151': {'1': {'151_15': [7514, 2279, 6251, 5140, 1500, 302, 301, 4495, 4522, 1934, 2310, 2311, 2312, 2313, 2314, 2315, 2094, 1406, 1407, 1409, 2095, 2096, 2097, 2099, 2101, 2102, 2726, 2727, 2105, 6140, 6144, 6145, 6146, 6147, 6149, 6150, 6143, 6011, 2133, 6244, 2134, 2135, 2164, 2165, 2166, 4545, 4552, 4554, 7016, 4619]}, '2': {'151_17': [4606, 7142, 7015, 4617, 4555, 4576, 3418, 3419, 4762, 4763, 6245, 2156, 6014, 6242, 6152, 6153, 6243, 6154, 6155, 6141, 6142, 2181, 2182, 2778, 2185, 7043, 1424, 2186, 2187, 2188, 2189, 1436, 3952, 2190, 4434, 2379, 2380, 2382, 5099, 2383, 1479, 7622, 297, 407, 2498, 2499, 2500, 2501, 7623, 2508, 2278]}}, '41A': {'1': {}, '2': {'41A_22': [4843, 4905, 7655, 4906, 4907, 4908, 4909, 4921, 4922, 4923, 4924, 4958, 3752, 3751, 3689, 7594, 3690, 6117, 5079, 3694, 3695, 3696, 3697, 3698, 3699, 3885, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 277, 288]}}, '25': {'1': {'25_272': [7391, 493, 494, 495, 400, 7588, 7392, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 2220, 2221, 2223, 2224, 3886, 3887, 3888, 3890]}, '2': {'25_269': [3890, 3891, 3949, 3950, 2229, 2230, 2231, 2233, 2232, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 4530, 748, 7390]}}, '70D': {'1': {'70D_70': [7571, 1644, 1605, 115, 37, 38, 39, 175, 5147, 4498, 1660, 1661, 1662, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1849, 1813, 1814, 4325, 3328, 3329, 3330, 3331, 3351, 3332, 3333]}, '2': {'70D_71': [3333, 3334, 3352, 3335, 3336, 3337, 3338, 3339, 3340, 1545, 1840, 1841, 4314, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 16, 5144, 1510, 7027, 156, 27, 28, 29, 4680, 90, 91, 7571]}}, '104': {'1': {'104_15': [1730, 1731, 1732, 1733, 1650, 1652, 5141, 1773, 1774, 1651, 4784, 608, 4791, 4792, 662, 1196, 663, 1197, 4473, 250, 251, 4597, 255, 218, 219, 220, 221, 222, 223, 224, 225, 226, 114, 1645, 1646, 7571]}, '2': {'104_16': [7571, 1644, 1605, 228, 229, 227, 230, 231, 232, 233, 242, 243, 253, 244, 245, 4474, 4790, 1220, 1221, 674, 4389, 4390, 530, 4785, 7607, 1764, 1765, 1766, 1767, 7129, 1744, 1745, 1746, 1747, 1748, 1749]}}, '7A': {'1': {'7A_85': [4962, 4725, 273, 405, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 3032, 3033, 3034, 3036, 3037, 3038, 3039, 3040, 3041, 2035, 2036, 2040, 3047, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210, 4981, 3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 4982]}, '2': {'7A_87': [3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 7472, 3230, 3232, 3233, 3234, 4462, 3235, 3236, 3237, 3238, 3239, 3240, 3068, 3069, 3070, 2041, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 7653, 3079, 3080, 3081, 3084, 469, 470, 471, 472, 473, 4705, 475, 476, 477, 478, 479, 480, 481, 482, 483, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495, 400, 281, 4, 4962]}}, '238': {'1': {'238_11': [7073, 7097, 7098, 7099, 7100, 7101, 7505, 5071, 5072, 4980, 2289, 1835, 1836, 1837, 2338, 661, 7026, 4323, 1824, 1825, 1826, 1827]}, '2': {'238_15': [1828, 1829, 1830, 1831, 4747, 2960, 1545, 1816, 1817, 1818, 1819, 1820, 2468, 4978, 4979, 4986, 7107, 7102, 7103, 7104, 7105, 7106, 6005, 1546, 7072]}}, '51X': {'1': {}, '2': {'51X_64': [7433, 4677, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2145, 4670, 3420, 3421, 3422, 3423, 4409, 1972, 1973, 1974, 1975, 1982, 2642, 7435, 4413, 7453, 1478, 1479, 315, 4495, 2809, 2810, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 4953]}}, '123': {'1': {'123_34': [6057, 2183, 2184, 2185, 1423, 1424, 1425, 1426, 1463, 4399, 4400, 1464, 1469, 1465, 1466, 1467, 1468, 1377, 1378, 4514, 4515, 1995, 1996, 1997, 1998, 1999, 2001, 2002, 1359, 274, 7164, 1505, 4509, 499, 500, 501, 1484, 1481, 7320, 1486, 1487, 1488, 1489, 1490]}, '2': {'123_36': [1491, 1492, 1493, 1482, 1494, 1495, 1496, 1515, 1499, 1485, 4518, 509, 510, 511, 513, 4513, 272, 335, 1278, 1934, 1935, 1937, 1938, 1939, 1940, 1941, 4516, 1369, 1370, 1371, 1454, 1455, 1453, 1456, 1457, 1458, 1459, 1460, 1462, 5149, 1419, 6094, 2102, 2103, 7154, 7153, 6056]}}, '65': {'1': {'65_74': [7564, 4521, 1283, 4456, 1284, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1121, 1122, 1123, 1124, 1125, 1127, 1130, 2550, 2551, 2553, 2554, 2555, 2556, 2557, 4436, 5008, 4640, 4347, 2349, 2351, 4445, 2352, 2573, 2574, 2576, 2577, 2578, 2579, 7395, 6124, 4004, 4005, 4006, 4007, 4008, 4010, 4009, 4011, 5110, 4012, 4013, 4014, 4098, 4015, 4017, 4018, 4019, 4020, 4100, 4021, 4022, 4023, 4024, 4027]}, '2': {'65_77': [5111, 4051, 4052, 4054, 4055, 4057, 4101, 4102, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4056, 4133, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 2583, 2584, 2585, 2586, 2587, 2588, 2589, 2590, 2591, 2592, 2357, 2358, 2359, 2360, 4348, 4646, 4647, 2346, 4435, 2594, 2595, 2596, 2597, 2599, 2600, 1155, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 1352, 1353, 1355, 1357, 1358, 7564]}}, '184': {'1': {'184_29': [4168, 4170, 4132, 4134, 4135, 4136, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 5139, 4214, 4215, 4216, 4289, 4290, 4291, 4222, 4224, 4225, 4226, 7462, 7461, 4227, 4228, 4229, 4230, 4232, 7351, 4234, 4235, 6019, 4236, 4650, 4651, 4652, 4653, 4654, 4655, 6138, 4656]}, '2': {'184_28': [4657, 7279, 4658, 4659, 4660, 4661, 4238, 4239, 4237, 3442, 4240, 4241, 4242, 4243, 4244, 4277, 4278, 4279, 4280, 7462, 7461, 4281, 4282, 4283, 4284, 4292, 4293, 4294, 4295, 4296, 4297, 4299, 4300, 4301, 4302, 5135, 4303, 4304, 4148, 4149, 3045, 4150, 4151, 4156, 4168]}}, '75': {'1': {'75_17': [4342, 4348, 4646, 4647, 4435, 2605, 2604, 2538, 2539, 2540, 2542, 6128, 2544, 2545, 2547, 2548, 946, 1131, 1132, 1133, 1134, 5131, 5132, 1305, 1306, 1307, 1308, 1309, 4332, 4333, 4334, 4712, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2863, 2864, 4486, 4586, 2872, 2873, 2874, 2877, 7394, 2878, 2879, 4460, 453, 4477, 3321, 4571, 2013, 2014, 4636, 2016, 2015, 2017, 2018, 2019, 2020, 2021, 2022, 4565, 4566, 4567, 2031, 2032, 2033, 2034, 2035, 2036, 2038]}, '2': {'75_19': [2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 4568, 4569, 7658, 4570, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 4637, 3322, 3563, 444, 445, 7393, 2880, 2882, 2883, 2885, 2886, 2887, 4587, 2842, 2843, 2844, 2845, 2846, 2847, 4397, 2848, 2849, 2850, 2851, 2852, 4707, 4708, 4336, 4337, 1317, 1318, 1319, 1281, 1329, 1330, 5129, 5130, 1138, 1139, 1140, 2515, 2516, 2517, 2518, 2520, 2521, 2522, 2524, 2523, 2532, 2533, 2534, 2535, 2536, 2616, 2617, 2557, 4436, 5008, 4640, 4347, 7181]}}, '46E': {'1': {}, '2': {'46E_94': [3085, 3034, 3114, 4353, 4355, 4354, 3196, 3345, 4727, 4728, 461, 2068, 2069, 2070, 2084, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 907, 908, 909, 786, 792, 320, 7491, 278, 2, 4726, 7555, 4962]}}, '61': {'1': {'61_106': [302, 340, 350, 2809, 2810, 2811, 843, 1013, 1015, 2812, 2813, 2814, 852, 853, 854, 855, 2815, 2816, 2817, 2818, 2819, 2820, 2822, 2823, 5032, 2867, 1035, 1314, 1316, 1315, 1317, 1318, 1319, 1281, 1329, 1330, 2919, 2920, 2921, 2922, 2923, 2924, 2925, 2948, 2949, 2950, 2954, 2955]}, '2': {'61_108': [7067, 2956, 2965, 2983, 2984, 7449, 2941, 2942, 2943, 2944, 2945, 2946, 2947, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1055, 2868, 2892, 2893, 2894, 2895, 2896, 2897, 2898, 3562, 2899, 884, 885, 886, 887, 2900, 2901, 2902, 2903, 1074, 6074, 5034, 793, 4529, 495, 400, 303]}}, '42D': {'1': {'42D_50': [7571, 92, 93, 94, 126, 6182, 322, 6312, 7197, 7195, 3653, 6055, 1073, 3641, 3642, 3643, 3583, 3584, 3585, 3625, 3626, 3627, 3628, 6052, 3613, 3615, 3594, 3595, 3596, 3597, 3598, 4465]}, '2': {'42D_51': [943, 3599, 3600, 3601, 3602, 3604, 3607, 6051, 6053, 7327, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 905, 4331, 4339, 6010, 7692, 3656, 7194, 6310, 6311, 112, 113, 114, 1646, 7571]}}, '45A': {'1': {'45A_60': [2037, 2040, 3047, 3202, 3203, 3204, 3205, 3206, 3241, 3242, 3243, 3244, 3245, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3534, 3535, 3536, 3537, 3538, 7556, 3539, 3540, 3541, 3542, 3543, 3139, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 7294, 4156, 4169, 4170, 4132, 4134, 4135, 4136, 4147, 5167, 4139, 4140, 7522, 4175, 5091, 5092, 6000]}, '2': {'45A_64': [4533, 4844, 4177, 4178, 4142, 4143, 4144, 5088, 5089, 4176, 4149, 3045, 4150, 4151, 2997, 4156, 4169, 4170, 4153, 4154, 4416, 4201, 4202, 4203, 5090, 4204, 4205, 4206, 3140, 3545, 3552, 3553, 3554, 3555, 3556, 3557, 3560, 7323, 3561, 3225, 3226, 3227, 3228, 3229, 7472, 3230, 3231, 3246, 3247, 7056, 3248, 3343, 3236, 3237, 3238, 3239, 3240, 3068, 3069, 3070, 4983]}}, '42': {'1': {'42_42': [1184, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 1199, 1200, 4385, 1201, 1202, 1203, 6115, 4849, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 3575, 3576, 3577, 3578, 3579, 3580, 3640, 3574, 3641, 3642, 3643, 3583, 3584, 3585, 3625, 3626, 3627, 3628, 6052, 3613, 3615, 3616]}, '2': {'42_44': [3605, 3606, 3607, 6051, 6053, 7327, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 7537, 905, 7538, 7539, 7540, 7541, 7542, 4331, 4339, 7616, 5081, 5082, 3639, 3644, 3645, 3646, 3647, 3661, 3573, 3662, 1212, 1213, 1214, 1215, 1216, 1217, 4563, 1218, 1270, 1272, 1273, 1274, 1275, 1276, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 1184]}}, '130': {'1': {'130_10': [7591, 7617, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 1751, 1752, 1753, 1754, 1759, 1758, 1750, 1769, 1770, 1771, 1760, 1761, 1762, 1763, 1764, 1765, 1767]}, '2': {'130_11': [1772, 1773, 1774, 1775, 1776, 1777, 1778, 1755, 1756, 1757, 1794, 7469, 1724, 1725, 1726, 1727, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 614, 615, 616, 617, 618, 619, 675, 620, 621, 293]}}, '67': {'1': {'67_4': [7391, 493, 494, 495, 400, 346, 317, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 3365, 3366, 3367, 3886, 3887, 3888, 7186, 7187, 3947, 3892, 3893, 3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3904, 3905, 3906, 3907, 3908, 3909, 5118, 3910, 3911, 3912, 3913, 3914, 7075, 3925, 3008, 6089]}, '2': {'67_6': [3917, 3918, 3926, 7074, 3919, 3920, 3921, 3922, 3923, 3112, 3924, 3927, 3928, 3929, 7108, 3935, 3936, 7110, 3938, 3939, 3940, 3941, 3943, 3944, 3945, 3946, 4001, 3949, 3950, 3372, 3373, 3374, 2233, 2232, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 2810, 7385]}}, '140': {'1': {'140_19': [7149, 7277, 6363, 6361, 6367, 4985, 6219, 6229, 6364, 1558, 4542, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 186, 187, 188, 189, 190, 191, 819, 264, 6059, 334, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1021, 6042, 1022, 1023, 1024, 895]}, '2': {'140_21': [896, 897, 893, 894, 1067, 1068, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 319, 7491, 274, 7, 194, 195, 196, 197, 198, 199, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1538, 4543, 1560, 1280, 6366, 6230, 6228, 7111, 6365, 6276, 6362, 7276, 7149]}}, '16C': {'1': {'16C_29': [7347, 3669, 7349, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 232, 233, 234, 235, 236, 237, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 262, 270]}, '2': {'16C_28': [5171, 7620, 7621, 2976, 2977, 2978, 2979, 2980, 2981, 2991, 2992, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1347, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1359, 320, 278, 7615]}}, '41': {'1': {'41_3': [288, 1171, 1172, 1173, 1174, 1175, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 1620, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 7348, 3669, 3671, 3672, 3674, 3675, 3676, 5073, 5074, 6054, 4330, 3679, 5075, 5076, 3682, 3864, 3865, 4910, 4911, 4912, 4913, 4914, 4915, 4957]}, '2': {'41_7': [4843, 4905, 7655, 4906, 4907, 4908, 4909, 3884, 3686, 5077, 5078, 3689, 7594, 3690, 6117, 5079, 3694, 3695, 3696, 3697, 3698, 3699, 3885, 3663, 7347, 3669, 7349, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 277, 288]}}, '114': {'1': {'114_5': [6200, 6201, 6196, 3532, 3533, 3491, 6007, 4349, 3184, 449, 450, 3181, 5142, 3187, 7669, 3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3198, 3199, 3200, 3201, 3085]}, '2': {'114_6': [3085, 3082, 3165, 3166, 3167, 3168, 3169, 3171, 3173, 3174, 3175, 3176, 3177, 3178, 3179, 4984, 448, 4457, 3182, 3183, 4350, 5143, 2833, 6006, 3493, 3494, 6197, 6198, 6199]}}, '38': {'1': {'38_9': [7333, 783, 784, 785, 786, 793, 7576, 401, 274, 2, 192, 795, 796, 57, 58, 59, 60, 1802, 1795, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 2961, 4901, 4902, 1672, 1673, 1674, 1675, 1849, 1851, 1813, 1814, 1816, 1817, 1818, 1819, 1822, 4994, 4995, 4996, 1826, 1827, 7382, 7381, 4742, 4743, 4744, 4745]}, '2': {'38_14': [4745, 4989, 4990, 4991, 4992, 1719, 1828, 1829, 4997, 4998, 4999, 1833, 1834, 1835, 1836, 1837, 2338, 1840, 1841, 4314, 1687, 1688, 1689, 4899, 4900, 4746, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1804, 1803, 77, 78, 79, 80, 82, 817, 818, 819, 265, 6059, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7370, 7339]}}, '7B': {'1': {'7B_91': [4962, 6059, 334, 406, 747, 842, 845, 846, 847, 848, 2795, 756, 757, 758, 759, 760, 761, 762, 763, 2007, 2008, 2009, 2010, 435, 7353, 7584, 3344, 7421, 7422, 6085, 2053, 2054, 2055, 4732, 3243, 3244, 3245, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3537, 3538, 7556, 3539, 3540, 3541, 3542, 3544]}, '2': {'7B_93': [3544, 3552, 3553, 3554, 3555, 3556, 3557, 7317, 3224, 3225, 3226, 3227, 3228, 3229, 7472, 3230, 3231, 3246, 3247, 7056, 4731, 2023, 2024, 2025, 6086, 7419, 7420, 3345, 4727, 4728, 461, 2068, 2069, 2070, 2084, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 907, 908, 909, 786, 792, 4529, 7586, 7587, 345, 320, 281, 4, 4962]}}, '18': {'1': {'18_3': [4359, 4360, 4361, 2202, 2203, 2204, 4888, 4799, 2205, 2687, 2688, 2689, 2696, 2697, 2700, 2701, 2702, 2784, 2785, 2786, 2787, 2181, 2182, 2778, 2185, 1421, 2317, 2318, 2319, 2464, 2465, 2466, 2467, 2494, 2495, 2496, 2497, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 4527, 2788, 2789, 2790, 903, 904, 779, 780, 781, 2796, 2797, 414, 415, 416, 417, 2800, 2801, 7599, 375]}, '2': {'18_4': [385, 2806, 2807, 2808, 486, 487, 2798, 2799, 753, 754, 755, 849, 850, 851, 2791, 7668, 2793, 2794, 1077, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2448, 5026, 2450, 2328, 2329, 2330, 2331, 2101, 2102, 2726, 2727, 2105, 2780, 2781, 2782, 2783, 2652, 2653, 2654, 2655, 2656, 2668, 2672, 2673, 2206, 2207, 2208, 2210, 2211, 2212, 7165, 4357]}}, '56A': {'1': {'56A_31': [395, 396, 397, 398, 399, 400, 4522, 1934, 2310, 2311, 2312, 2313, 2314, 2315, 2094, 1406, 1407, 1409, 2095, 2096, 2097, 2099, 2101, 2102, 2726, 2727, 2105, 2728, 2729, 2730, 6083, 2733, 2734, 5169, 2735, 2736, 2725, 2737, 2738, 2739, 7177, 2740, 2741, 2742, 2743, 2745, 2746, 2747, 2748, 7062, 6075, 2631, 2359, 2360, 4340]}, '2': {'56A_30': [4344, 2349, 2351, 2621, 2622, 2750, 2751, 2752, 2753, 2754, 2755, 2756, 2764, 7178, 2765, 2766, 2767, 2772, 2768, 2769, 2770, 2771, 2773, 2774, 2777, 2775, 2181, 2182, 2778, 2185, 7043, 1424, 2186, 2187, 2188, 2189, 1436, 3952, 2190, 4434, 2379, 2380, 2382, 5099, 2383, 2003, 1358, 340, 350, 351, 352, 353, 354]}}, '40E': {'1': {'40E_90': [7672, 826, 827, 71, 72, 5144, 7009, 1604, 1573, 1574, 1575, 4583, 1576, 1577, 1578, 4364, 1579, 7673, 7674, 7675, 7676, 7677, 7560, 4986, 7107, 7102, 7103, 7104, 7105, 7106, 6005, 7072]}, '2': {'40E_91': [7073, 7097, 7098, 7099, 7100, 7101, 7505, 7678, 7559, 7679, 7680, 7681, 7682, 7683, 1582, 4793, 6068, 1584, 1585, 1586, 1600, 7010, 5147, 68, 30, 69, 7686]}}, '41B': {'1': {}, '2': {}}, '25X': {'1': {'25X_10': [4953, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 6058, 791, 7576, 4529, 7585, 7587, 345, 318, 312, 1444, 1445, 4407, 2637, 2721, 7012, 2201, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 3365, 3366, 3367, 3886, 3888, 3890]}, '2': {'25X_11': [3890, 3891, 3949, 3950, 3372, 3373, 3374, 2233, 2232, 2234, 2235, 2239, 2241, 2242, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 4495, 2809, 2810, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 4953]}}, '17A': {'1': {'17A_15': [4747, 7298, 2294, 6039, 1816, 6171, 4974, 4976, 2510, 7242, 7238, 7224, 1581, 4793, 6068, 1584, 983, 984, 985, 987, 988, 138, 139, 140, 141, 142, 143, 92, 93, 94, 126, 989, 6009, 990, 991, 992, 7044, 993, 5060, 5061, 1235, 1251, 253, 7130, 1237, 1253, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1012]}, '2': {'17A_17': [956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 1252, 1228, 1244, 4597, 1246, 1229, 5174, 6078, 7112, 972, 973, 974, 975, 5055, 976, 127, 112, 113, 128, 129, 130, 131, 132, 7458, 979, 980, 981, 1576, 1577, 1578, 4364, 1579, 1580, 7237, 7241, 7247, 7243, 7334, 6274, 1838, 2338, 2290, 6179, 6180, 7297, 7389, 7025]}}, '32X': {'1': {'32X_77': [3640, 3574, 3641, 3642, 3643, 3583, 3584, 3585, 3587, 3588, 3589, 3605, 3606, 3607, 3608, 3609, 3598, 4465, 944, 945, 947, 948, 949, 928, 588, 590, 1729, 1731, 1738, 614, 675, 621, 6207, 4495, 2809, 2810, 842, 846, 848, 759, 760, 761, 762, 763, 4952]}, '2': {'32X_76': [4952, 768, 770, 773, 774, 906, 909, 786, 791, 6058, 790, 7576, 4529, 7585, 7587, 496, 7490, 522, 7659, 1741, 1747, 1749, 544, 546, 914, 938, 940, 941, 942, 943, 3599, 3610, 3611, 6135, 3612, 3613, 3615, 3616, 3618, 3619, 6087, 3620, 3621, 3622, 3623, 4387, 3634, 3635, 3636, 905, 4331, 4339, 7616, 5081, 5082]}}, '79A': {'1': {'79A_27': [326, 1443, 1444, 1445, 4407, 2637, 2638, 2721, 7012, 2644, 2645, 2646, 2647, 2648, 2649, 2650, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2909, 6020, 6023, 6026, 6027, 6029]}, '2': {'79A_28': [6030, 6028, 6025, 6024, 6021, 7507, 2663, 2664, 2665, 2666, 2667, 2696, 2697, 2700, 2701, 2702, 2703, 2704, 2706, 6070, 2708, 5134, 2718, 7435, 2722, 4413, 1476, 7453, 1480, 2912, 1358, 326]}}, '185': {'1': {'185_53': [4167, 4170, 4153, 4154, 4183, 4184, 4185, 4584, 4186, 4187, 4188, 4189, 4190, 4097, 4103, 4104, 4105, 4106, 6139, 4108]}, '2': {'185_56': [4108, 4109, 4110, 4111, 4112, 4113, 4095, 4192, 4194, 4196, 4422, 7369, 7295, 4199, 4131, 7294, 4156, 4167]}}, '220': {'1': {'220_10': [4686, 4687, 102, 6239, 104, 6241, 105, 4365, 4367, 4788, 6273, 136, 137, 193, 979, 1529, 1530, 1531, 1532, 1592, 1593, 1594, 1596, 4363, 1598, 1597, 1599, 1600, 1601, 1602, 1603, 1604, 1573, 1574, 1575, 4583, 1576, 1577, 1578, 4364, 1580, 7237, 7241, 7247, 7243, 7126, 7124, 7343, 7318, 4980, 2289, 1835, 1836, 1837, 2338, 2290, 6179, 6180, 7297, 1813, 1814, 661, 7026, 1882, 1856, 1857, 1858, 1859, 1860, 4492, 1861, 1862, 1863, 1864, 1865, 1866, 1850, 1867, 4489, 4323, 1824, 1825, 1826, 1827]}, '2': {'220_12': [1828, 1829, 1830, 1831, 4464, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 4493, 4362, 4747, 2960, 1545, 1840, 1841, 1842, 7298, 2294, 6039, 1816, 1817, 1818, 1819, 1820, 2468, 7319, 7342, 7125, 7344, 2510, 7242, 7238, 7224, 1581, 4793, 6068, 1584, 1585, 1586, 1600, 1601, 1602, 1603, 1604, 1587, 4539, 1606, 1607, 1608, 1609, 1610, 7471, 1512, 1538, 1513, 4592, 4375, 4366, 4479, 4376, 4377, 4890, 4369, 4368, 4781, 107, 7092, 110, 124, 4686]}}, '33X': {'1': {}, '2': {}}, '59': {'1': {'59_10': [2037, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 4830, 3091, 7667, 3092, 3093, 3094, 3095, 3096, 3097, 3098, 7652]}, '2': {'59_11': [3099, 3100, 3101, 3102, 3103, 5105, 3105, 3106, 3107, 3108, 7666, 3111, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 4983]}}, '118': {'1': {}, '2': {}}, '84A': {'1': {'84A_1': [7188, 3082, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 7638, 3130, 3131, 3132, 3133, 5046, 7639, 3134, 7640, 3135, 3136, 3138, 3139, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 7294, 4156, 4168]}, '2': {'84A_3': [4168, 4170, 4153, 4154, 4416, 4201, 4202, 4203, 4204, 4205, 4206, 3140, 3141, 3142, 3143, 3144, 5047, 4678, 7639, 3146, 3147, 7640, 3148, 3149, 3150, 3151, 3153, 3152, 2995, 3154, 3155, 3156, 3158, 3159, 3160, 3161, 3162, 3163, 3164, 7653, 3079, 3080, 3081, 7188]}}, '66': {'1': {'66_17': [7391, 493, 494, 495, 400, 346, 317, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 3365, 3366, 3367, 3886, 3887, 3888, 7186, 7187, 3954, 3955, 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964, 3965, 3968, 3966, 5113, 3918, 5114]}, '2': {'66_18': [3980, 6089, 3981, 3982, 3983, 3985, 3986, 3987, 3989, 3990, 3991, 3998, 3999, 4000, 4001, 3949, 3950, 3372, 3373, 3374, 2233, 2232, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 2810, 7387]}}, '14': {'1': {'14_15': [248, 249, 250, 251, 252, 255, 234, 235, 236, 1190, 1191, 1192, 1193, 1194, 1195, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 4415, 301, 336, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 2914, 2989, 2990, 1042, 1043, 1044, 1045, 4710, 1046, 1047, 1048, 1049, 4333, 4334, 4712, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2863, 2864, 4486, 2865, 2866, 6044]}, '2': {'14_16': [6041, 2826, 2842, 2843, 2844, 2845, 2846, 2847, 4397, 2848, 2849, 2850, 2851, 2852, 4707, 4708, 4336, 1051, 1052, 1053, 1054, 1036, 4709, 1037, 1038, 2998, 2999, 3000, 3001, 3002, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 297, 496, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1185, 1186, 1187, 1188, 216, 217, 242, 243, 244, 245, 246, 248]}}, '68X': {'1': {}, '2': {'68X_88': [6185, 6186, 6216, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3410, 3407, 3408, 3409, 4558, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 4762, 4763, 6245, 2156, 6014, 6242, 6152, 6153, 6243, 6154, 6155, 1987, 1989, 1995, 1998, 2002, 1358, 7665]}}, '7': {'1': {'7_72': [4962, 4725, 273, 405, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 3032, 3033, 3034, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 2035, 2036, 2040, 3047, 3202, 3203, 3204, 7641, 7642, 7643, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 5046, 7639]}, '2': {'7_70': [7639, 7640, 5047, 3224, 3225, 3226, 3227, 3228, 3229, 7644, 7645, 7646, 3238, 3239, 3240, 3068, 3069, 3070, 2041, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 7653, 6333, 3084, 469, 470, 471, 472, 473, 4705, 475, 476, 477, 478, 479, 480, 481, 482, 483, 485, 487, 488, 489, 490, 491, 492, 493, 494, 495, 400, 281, 4, 4962]}}, '38D': {'1': {'38D_68': [7333, 783, 784, 785, 786, 793, 7576, 401, 274, 2, 192, 795, 796, 57, 58, 59, 60, 1802, 1795, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 4744, 4745]}, '2': {'38D_61': [4745, 4989, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1804, 1803, 77, 78, 79, 80, 82, 817, 818, 819, 265, 6059, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7370, 7339]}}, '32': {'1': {'32_57': [621, 289, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 525, 526, 527, 528, 529, 530, 531, 709, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 910, 911, 912, 913, 914, 938, 940, 941, 942, 943, 3599, 3600, 3601, 3602, 3604, 3616, 3618, 3619, 3620, 3622, 3623, 4387]}, '2': {'32_58': [3624, 3587, 3588, 3590, 3589, 3591, 3592, 3593, 3594, 3595, 3596, 3597, 3598, 4465, 944, 945, 947, 948, 949, 928, 929, 930, 931, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 7569]}}, '84X': {'1': {'84X_57': [342, 4495, 746, 844, 846, 848, 758, 760, 2007, 4571, 2014, 2016, 2017, 7362, 3134, 3135, 4188, 7363, 4588, 4589, 7364, 7365, 4210, 4211, 4215, 4216, 4222, 4224, 4225, 7462, 7461, 4230, 4248, 4252, 4305, 4306, 4307]}, '2': {'84X_62': [4307, 4308, 4253, 4268, 4273, 4277, 7462, 7461, 4281, 4282, 4283, 4284, 4295, 4296, 4301, 4302, 4591, 4590, 7366, 7367, 7368, 3143, 3144, 3145, 5128, 2060, 2062, 2064, 4727, 768, 772, 775, 906, 909, 786, 6058, 792, 4529, 7586, 7587, 303]}}, '15D': {'1': {'15D_62': [395, 396, 397, 398, 399, 400, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 2914, 2915, 2916, 2917, 2918, 1300, 1301, 1302, 1303, 5129, 5130, 1135, 7064, 7066, 2924, 2925, 2948, 2949, 2950, 2954, 2955]}, '2': {'15D_63': [7067, 2956, 2965, 2983, 2984, 2941, 2942, 7065, 7063, 1137, 5131, 5132, 1331, 1332, 1333, 1334, 2985, 2986, 2988, 3001, 3002, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 340, 407, 2498, 2499, 7216, 7558, 353, 354]}}, '68A': {'1': {'68A_86': [4495, 1278, 1282, 4456, 1284, 1285, 1286, 1288, 1289, 1360, 1362, 1363, 1364, 1365, 1366, 1367, 1369, 1370, 1371, 1372, 1373, 1374, 1375]}, '2': {'68A_87': [1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1358, 7665]}}, '145': {'1': {'145_102': [4320, 1476, 7453, 1478, 1479, 7622, 334, 406, 747, 842, 845, 846, 847, 848, 2795, 756, 757, 758, 759, 760, 761, 762, 763, 2007, 2008, 2009, 2010, 435, 7353, 4571, 2013, 2014, 4636, 2016, 2015, 2017, 3259, 7361, 7362, 3129, 7638, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3138, 3139, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 7294, 4132, 4171, 4172, 4173, 2962, 2963, 2964, 4175, 5091, 5092, 7574]}, '2': {'145_105': [7574, 4177, 4178, 4179, 2993, 2994, 4180, 4181, 4182, 4151, 2997, 4153, 4154, 4416, 4201, 4202, 4203, 5090, 4204, 4205, 4206, 3140, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3148, 5127, 5128, 2996, 3258, 2060, 2061, 2062, 2063, 2064, 2065, 4727, 4728, 461, 2068, 2069, 2070, 2084, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 907, 908, 909, 786, 792, 4529, 7586, 7587, 7588, 345, 325, 1443, 1444, 1445, 4407, 4320]}}, '76': {'1': {'76_36': [2243, 2200, 5006, 5007, 2655, 2656, 2668, 2672, 2673, 2206, 4798, 4795, 2675, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2158, 2159, 4409, 2173, 2174, 3425, 2763, 2619, 2620, 2345, 5008, 4640, 4347, 4341]}, '2': {'76_37': [4341, 4348, 4646, 4647, 2363, 2364, 2618, 5011, 2762, 2110, 2111, 2112, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2123, 2685, 4795, 2686, 7510, 4799, 2205, 2687, 2688, 2689, 2696, 2697, 4414, 5009, 2199]}}, '27B': {'1': {'27B_23': [300, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1244, 254, 1246, 1229, 1231, 1232, 1233, 1234, 7358, 5174, 6078, 7112, 972, 1625, 1626, 1627, 1628, 1629, 6234, 332]}, '2': {'27B_34': [324, 7479, 7484, 7485, 7486, 7487, 1634, 1635, 1636, 1637, 1638, 992, 7044, 993, 5060, 5061, 1231, 1232, 1233, 1234, 7358, 1235, 1251, 253, 5033, 7130, 1237, 1238, 1239, 247, 1240, 1241, 1242, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 4415, 303]}}, '69X': {'1': {'69X_59': [4495, 4720, 1443, 1444, 1445, 1447, 2637, 1956, 2109, 3427, 3428, 3429, 3432, 3434, 3435, 3453, 3437, 3440, 3441, 4560, 5126]}, '2': {'69X_60': [4564, 3444, 3447, 3454, 3455, 5022, 3461, 4537, 2176, 1982, 7435, 4413, 1480, 1358, 7665]}}, '27': {'1': {'27_19': [4595, 4596, 4563, 1218, 1258, 1259, 1260, 6130, 1262, 4692, 1245, 4693, 1254, 1255, 1256, 4694, 1249, 1247, 1248, 1273, 1274, 1275, 1276, 1277, 1219, 1220, 1221, 664, 665, 666, 667, 668, 614, 615, 616, 617, 618, 619, 675, 1500, 302, 4495, 4522, 1934, 2310, 2311, 2312, 2313, 2314, 2315, 2094, 1406, 1407, 1409, 2095, 2096, 2097, 2099, 2101, 2102, 2103, 2332, 2333, 2334, 4662, 2335, 2336, 2337, 2339, 2340, 2601, 4446, 2603, 2557, 4436, 5008, 4640, 4347, 2349, 2351, 2621, 2622, 2623, 2624, 4441, 2626, 2669, 4641, 4642, 2355]}, '2': {'27_17': [2353, 4643, 4644, 4645, 4440, 2628, 2629, 6095, 7062, 6075, 2631, 2359, 2360, 4348, 4646, 4647, 2346, 4435, 2632, 2633, 2369, 2370, 2371, 2372, 2373, 2377, 2378, 2183, 2184, 2185, 7043, 1424, 2186, 2187, 2188, 2189, 1436, 3952, 2190, 4434, 2379, 2380, 2382, 5099, 2383, 2002, 1358, 298, 496, 7490, 497, 515, 516, 4384, 519, 521, 522, 7659, 523, 669, 670, 671, 672, 4382, 1196, 1197, 1198, 1199, 1200, 4385, 1201, 4695, 1243, 4696, 4697, 1250, 4698, 4383, 4699, 4388, 4700, 1266, 1267, 1268, 1269, 1265, 6115, 4849, 4593, 4595]}}, '63': {'1': {'63_26': [2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2053, 2054, 2055, 7636, 7651, 7635, 3254, 5127, 5128, 3126, 3255, 3309, 3306, 4772, 3276, 7372, 5106, 4396, 4773, 4393, 4395, 4313, 7360, 3277, 3278, 3279, 3478, 3280, 3282, 3283]}, '2': {'63_30': [3283, 3284, 3285, 7324, 7360, 5106, 4396, 4773, 4393, 4395, 4313, 7373, 3289, 3312, 4771, 3307, 3262, 3153, 7362, 3129, 7634, 3249, 7637, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2039]}}, '49': {'1': {'49_27': [342, 4521, 1934, 2385, 2386, 2387, 2388, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1121, 1122, 1123, 1124, 1125, 1127, 1130, 1140, 2515, 2516, 2517, 2518, 4749, 4750, 4751, 4754, 4755, 3003, 2522, 2524, 2523, 2532, 4877, 4878, 7180, 7123, 4347, 7181]}, '2': {'49_28': [4342, 4348, 7122, 7121, 7183, 4880, 2539, 2540, 2542, 3004, 4756, 3005, 4757, 4758, 6343, 4759, 4760, 2547, 2548, 946, 1155, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1347, 2634, 2635, 2636, 5097, 2383, 2002, 1358, 5192, 342]}}, '25D': {'1': {'25D_277': [7391, 493, 494, 495, 400, 7588, 7392, 312, 1444, 1445, 4407, 2637, 2721, 7012, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 7184, 2225, 4620, 7563, 7231, 7128, 7117, 7118, 6289]}, '2': {'25D_275': [6290, 7120, 7119, 7127, 7143, 7168, 4616, 2227, 7049, 7627, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 347, 406, 4530, 748, 7390]}}, '33A': {'1': {'33A_22': [3677, 3678, 3679, 3711, 3712, 3713, 3714, 3715, 5175, 3755, 3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765, 3766, 3767, 3768, 3770, 3769, 3771, 3772, 3773, 3774, 7172, 3775, 3777, 3778, 3781, 3782, 3783, 3784, 3785, 3786, 3787, 3788, 3789, 3790, 3791, 3792, 3793, 3794, 3795, 3797, 3798, 3799, 6076, 6077, 3569, 3571, 3801, 3802, 3805, 7175, 3806, 3807, 3808, 3809, 7174]}, '2': {'33A_25': [6048, 7173, 3811, 3812, 3664, 3813, 3814, 7592, 3815, 3816, 3566, 3567, 7292, 3818, 3819, 3820, 3822, 3823, 3824, 3565, 3825, 3826, 3827, 3828, 3829, 3831, 3832, 3833, 3834, 3835, 3836, 3837, 3838, 3840, 3842, 3843, 3844, 3845, 3846, 3848, 3849, 3850, 3851, 3852, 3853, 3854, 3855, 3856, 3857, 3858, 3859, 3860, 3861, 3862, 3863, 3748, 6050, 3749, 3750, 3752, 3751, 3689, 3692]}}, '31D': {'1': {'31D_50': [7571, 1190, 1191, 1192, 1193, 1194, 1195, 674, 4389, 4390, 530, 531, 709, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 910, 911, 912, 913]}, '2': {'31D_51': [928, 929, 930, 931, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 4791, 4792, 1185, 1186, 1187, 1188, 237, 206, 1643, 7571]}}, '40': {'1': {'40_27': [7132, 1558, 2957, 2958, 4938, 1549, 1523, 1524, 1525, 1526, 6080, 1585, 1586, 1587, 4539, 1606, 1607, 1608, 1609, 1610, 7471, 1533, 1534, 1535, 1536, 1537, 186, 838, 839, 840, 841, 49, 51, 52, 261, 270, 335, 4521, 1935, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 2642, 2643, 2644, 2709, 2710, 2711, 2712, 2655, 2656, 2668, 2672, 2673, 2206, 4798, 2674, 2675, 2124, 2125, 2126, 2127, 2128, 2129, 4797, 7473, 2681, 4691, 2682, 4796, 4690, 4411, 4715]}, '2': {'40_31': [4795, 4688, 4689, 4410, 2678, 2679, 2680, 2676, 2118, 2119, 2120, 2121, 2123, 2685, 2686, 7510, 4799, 2205, 2687, 2688, 2689, 2696, 2697, 2713, 2714, 2715, 2716, 2718, 2719, 1989, 1990, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2001, 2002, 1358, 319, 279, 7, 11, 14, 15, 834, 835, 836, 837, 1506, 1507, 1508, 1509, 1510, 1511, 1592, 1593, 1594, 1595, 1596, 4363, 1598, 1597, 1599, 1573, 1574, 1575, 4583, 1518, 1519, 1520, 7661, 1521, 4546, 1559, 987, 4937, 1547, 1548, 1280, 6348]}}, '83A': {'1': {'83A_17': [324, 7500, 7501, 7502, 7503, 7504, 1555, 1556, 1557, 1181, 7350, 6230, 6228, 4788, 6273, 4891, 7458, 7425, 7424, 7423, 1566, 1567, 173, 174, 175, 176, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 1614, 1615, 1478, 1479, 315, 336, 1279, 1283, 4456, 1284, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2448, 5026, 2450, 2490, 2491, 2492]}, '2': {'83A_23': [2492, 2493, 2465, 2466, 2467, 2494, 2495, 2496, 2497, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 1170, 1069, 1070, 1071, 4528, 1072, 1352, 1353, 1355, 1357, 1359, 319, 1443, 1616, 1617, 1618, 1619, 195, 196, 197, 198, 199, 200, 201, 202, 151, 153, 154, 155, 156, 157, 158, 159, 1539, 1540, 7426, 7427, 7428, 138, 133, 134, 4890, 4369, 6219, 6229, 6348, 1182, 1550, 1551, 1552, 6234, 332]}}, '9': {'1': {'9_5': [7132, 6230, 6228, 4788, 6273, 136, 137, 193, 138, 167, 168, 169, 141, 142, 143, 115, 37, 38, 39, 40, 146, 147, 184, 185, 186, 187, 188, 189, 190, 191, 819, 263, 277, 336, 1278, 1282, 4456, 1284, 1285, 1286, 1288, 1289, 1290, 1291, 2389, 2390, 2391, 2392, 2393, 2394, 1090, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2452, 2453, 2454, 2455, 2456, 2445, 2457, 4392]}, '2': {'9_7': [4392, 2446, 2459, 2460, 2461, 2462, 2463, 2431, 2432, 2433, 2434, 2435, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 1344, 1345, 1347, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1359, 319, 281, 4, 194, 195, 196, 197, 198, 199, 200, 201, 148, 149, 150, 27, 28, 29, 4680, 90, 91, 129, 130, 131, 164, 165, 166, 7458, 4479, 4376, 4377, 4890, 4369, 6219, 6229, 7131]}}, '11': {'1': {'11_40': [6122, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 261, 271, 404, 747, 842, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 441, 442, 443, 444, 445, 4779, 7057, 4847, 4848, 449]}, '2': {'11_42': [449, 450, 3181, 451, 447, 5045, 4460, 453, 454, 4475, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 875, 876, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 903, 904, 906, 907, 908, 909, 786, 793, 7576, 401, 320, 278, 7615, 10, 12, 14, 15, 17, 18, 19, 21, 7602, 22, 23, 24, 25, 27, 28, 29, 4680, 31, 32, 33, 7220, 6122]}}, '84': {'1': {'84_27': [7188, 3082, 7660, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 7638, 3130, 3131, 3132, 3133, 5046, 7639, 3134, 7640, 3135, 3136, 3138, 3139, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 7294, 4156, 4168, 4170, 4132, 4134, 4135, 4136, 4207, 4208, 4209, 4210, 4211, 4212, 4213, 5139, 4214, 4215, 4216, 4217, 4218, 4219, 4220, 4221, 4222, 4224, 4225, 4226, 4227, 4228, 4229, 4230, 4231, 7352, 4246, 4248, 4249, 4250, 4251, 4252, 4253, 4254, 6137, 4256, 4257, 4258, 4259, 7272, 4956]}, '2': {'84_30': [4260, 4261, 7271, 5136, 4262, 4263, 4264, 7296, 4265, 4266, 4267, 4268, 4269, 4270, 4271, 4273, 4274, 4275, 4276, 4277, 4278, 4279, 4280, 4281, 4282, 4283, 4284, 4285, 4286, 4287, 4288, 4295, 4296, 4297, 4299, 4300, 4301, 4302, 5135, 4303, 4304, 4148, 4149, 3045, 4150, 4151, 4156, 4168, 4170, 4153, 4154, 4416, 4201, 4202, 4203, 4204, 4205, 4206, 3140, 3141, 3142, 3143, 3144, 4678, 5047, 3146, 3147, 3148, 3149, 3150, 3151, 3153, 3152, 2995, 3154, 3155, 3156, 3158, 3159, 3160, 3161, 3162, 3163, 3164, 7653, 7188]}}, '40B': {'1': {'40B_63': [6004, 7, 11, 14, 15, 834, 835, 836, 837, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1538, 4548, 980, 981, 1518, 1519, 1520, 7661, 1521, 4546, 1559, 1560, 1280, 6348, 1182, 1550, 1551, 1552, 1553, 7023, 7014, 4315, 4316]}, '2': {'40B_65': [4316, 4317, 7013, 6162, 7022, 1554, 1555, 1556, 1557, 1181, 7350, 1558, 1549, 1523, 1524, 1525, 1526, 6080, 983, 984, 985, 1529, 4542, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 186, 838, 839, 840, 841, 49, 51, 52, 4725, 6071]}}, '31A': {'1': {'31A_25': [621, 289, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 525, 526, 527, 528, 529, 530, 531, 709, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 557, 558, 559, 560, 562, 563, 564, 565, 566, 707, 708, 731, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 680, 681, 682, 683, 684, 685, 686]}, '2': {'31A_26': [686, 688, 689, 690, 5062, 691, 692, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 7569]}}, '46A': {'1': {'46A_74': [807, 808, 809, 810, 811, 812, 813, 814, 81, 817, 818, 819, 264, 6059, 334, 406, 747, 842, 845, 846, 847, 848, 2795, 756, 757, 758, 759, 760, 761, 762, 763, 2007, 2008, 2009, 2010, 435, 7353, 4571, 2013, 2014, 4636, 2016, 2015, 2017, 2018, 2019, 2020, 2021, 2022, 4565, 4566, 4567, 2031, 2032, 2033, 2034, 2035, 2036, 2039]}, '2': {'46A_67': [2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 4568, 4569, 7658, 4570, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 4727, 4728, 461, 2068, 2069, 2070, 2084, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 906, 907, 908, 909, 786, 792, 401, 320, 7491, 278, 2, 192, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 7513, 806, 807, 7688, 7689]}}, '25A': {'1': {'25A_273': [7391, 493, 494, 495, 400, 7588, 7392, 312, 1444, 1445, 4407, 2637, 2721, 7012, 7566, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 4601, 5058, 4602, 4603, 4604, 4605, 4562, 7386, 4607, 4608, 4609, 4610, 4611, 4612, 4613, 3370, 7168, 4616]}, '2': {'25A_270': [4620, 7563, 3368, 4626, 4627, 4624, 4625, 4628, 4623, 4619, 4561, 4629, 4630, 4631, 4632, 5164, 4633, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 7626, 7435, 2722, 4413, 1476, 7453, 1478, 1479, 315, 347, 406, 4530, 748, 7390]}}, '33D': {'1': {'33D_61': [407, 2498, 2499, 2500, 2501, 7623, 3714, 3719, 3718, 3720, 3721, 3722, 5069, 3723, 3724, 3725, 3726, 7039, 7204, 3728, 3729, 3730, 3732]}, '2': {'33D_62': [3732, 3733, 3734, 3735, 7039, 7204, 3736, 3738, 7170, 3740, 3741, 5165, 3743, 3780, 3744, 3745, 3746, 3749, 3750, 6252, 302, 333, 747, 748]}}, '16D': {'1': {'16D_30': [7347, 3669, 7349, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 1641, 1642, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 262, 270, 336, 1279, 1282, 4456, 1284, 1285, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1320, 1321, 1322, 1323, 1324, 7068, 2966, 2967, 2968, 2969, 2970, 2971, 5051, 5052, 2975]}, '2': {}}, '29A': {'1': {'29A_14': [289, 7617, 496, 7490, 515, 516, 4384, 519, 521, 522, 7659, 523, 525, 526, 527, 528, 529, 530, 531, 709, 532, 649, 650, 651, 4472, 4386, 653, 722, 723, 724, 725, 726, 727, 728, 729, 730, 5160, 5155, 4806, 5156, 7552, 924, 925, 927, 938]}, '2': {'29A_15': [950, 915, 916, 917, 918, 919, 7551, 4800, 5157, 5154, 4804, 738, 739, 740, 741, 742, 743, 744, 745, 600, 654, 655, 656, 657, 658, 659, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 621, 291]}}, '16': {'1': {'16_20': [7347, 3669, 7349, 1631, 1632, 5053, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 231, 232, 233, 234, 235, 236, 237, 213, 214, 4432, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 262, 270, 336, 1279, 1282, 4456, 1284, 1285, 1287, 7580, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1320, 1321, 1322, 1323, 1324, 7068, 2966, 2967, 2968, 2969, 2970, 2971, 5051, 5052, 2975]}, '2': {'16_24': [5171, 7620, 7621, 2976, 2977, 2978, 2979, 2980, 2981, 2991, 2992, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 7293, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1347, 1348, 1349, 1350, 1352, 1353, 7589, 1355, 1357, 1359, 320, 278, 7615, 8, 12, 14, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 215, 216, 217, 218, 219, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 7347]}}, '13': {'1': {'13_60': [6235, 327, 6016, 6017, 7192, 4683, 107, 6125, 7092, 110, 112, 113, 114, 115, 37, 38, 39, 116, 117, 118, 119, 44, 7603, 45, 46, 47, 48, 49, 51, 52, 261, 270, 335, 4521, 1935, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 4379, 1963, 4549, 1964, 4674, 4663, 4664, 2160, 2161, 4665, 4666, 4667, 4668, 4677, 2136, 2137, 7201, 2138, 2139, 2140, 2141, 2142, 2143, 2145, 4670, 2165, 2166, 4545, 4552, 4554, 7229]}, '2': {'13_67': [7229, 4617, 4555, 4576, 3418, 3419, 2374, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 6245, 4671, 4672, 3465, 4673, 2172, 2159, 4409, 4550, 4551, 1965, 1966, 1975, 1977, 1978, 1980, 1981, 1982, 4406, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2001, 2002, 1359, 319, 281, 7, 11, 14, 15, 17, 18, 19, 21, 7602, 85, 86, 87, 88, 89, 27, 28, 29, 4680, 90, 91, 92, 93, 94, 102, 6239, 104, 6241, 105, 6253, 7191, 6238, 6182, 322, 323, 6234]}}, '38A': {'1': {'38A_20': [7333, 783, 784, 785, 786, 793, 7576, 401, 274, 2, 192, 795, 796, 57, 58, 59, 60, 1802, 1795, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 1813, 1814, 1816, 1817, 1818, 1819, 1822, 4994, 4995, 4996, 1826, 1827, 7382, 7381, 4742, 4743, 4745]}, '2': {'38A_32': [4745, 4989, 4990, 4991, 4992, 1719, 1828, 1829, 4997, 4998, 4999, 1833, 1834, 1835, 1836, 1837, 2338, 1840, 1841, 1842, 7374, 1845, 7166, 1847, 1696, 1697, 1698, 1699, 1700, 1701, 1702, 1703, 1905, 1804, 1803, 77, 78, 79, 80, 82, 817, 818, 819, 265, 6059, 349, 404, 747, 748, 749, 750, 751, 752, 753, 7370, 7339]}}, '15A': {'1': {'15A_83': [395, 396, 397, 398, 399, 400, 7581, 1283, 7579, 1285, 1016, 1017, 1018, 1019, 1020, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1101, 1102, 1103, 1104]}, '2': {'15A_84': [1105, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 2437, 1117, 1118, 1119, 1120, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1069, 1070, 1071, 4528, 1072, 7577, 1353, 1354, 7589, 7578, 7582, 340, 407, 2498, 2499, 7216, 7558, 353, 354]}}, '53': {'1': {'53_20': [1184, 496, 7490, 497, 515, 516, 4384, 518, 2255, 2256, 2717, 2257, 2258, 2259, 2260, 2261, 4469, 2262, 2264, 2265, 2269, 2270]}, '2': {'53_21': [2270, 2274, 7601, 2508, 2278, 2279, 2720, 2280, 2281, 2282, 4468, 2283, 616, 617, 618, 619, 675, 4380]}}, '66B': {'1': {'66B_58': [7391, 493, 494, 495, 400, 346, 317, 312, 1444, 1445, 7078, 1449, 1450, 1451, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 7239, 4401, 2213, 2214, 2215, 2216, 4599, 2217, 2218, 2219, 3365, 3366, 3367, 3886, 3887, 3888, 7186, 7187, 3954, 3955, 3956, 3957, 3958, 3959, 7234, 4002, 5112, 4003]}, '2': {'66B_59': [4003, 4404, 4405, 3991, 3998, 3999, 4000, 4001, 3949, 3950, 3372, 3373, 3374, 2233, 2232, 2234, 2235, 2236, 2237, 2238, 5056, 2239, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 1472, 4408, 1473, 1474, 1476, 7453, 1478, 1479, 315, 347, 406, 2810, 7387]}}}

In [93]:
date = "2020-07-21"

In [94]:
len(list(dictionary.keys()))

130

In [138]:
keep_variations = {}
not_operating = []
count = 0
count1 = 0
problematic_routes = 0
exceptions = []
beginning_subroutes = 0

for key, value in dictionary.items():
    
    route = key
    for key1, value1 in dictionary[route].items():
        direction = key1
        for key2, value2 in dictionary[route][direction].items():
            subroute = key2
            old_stops = value2
            route_id = get_route_id(route)
            day_of_week = get_weekday(date)
            name_day_of_week = get_name_day_of_week(day_of_week)
            date_int = get_integer_date(date)
            exception_code = is_exception(date_int)
            service_id = get_service_id(date_int, name_day_of_week, exception_code )
            trip_ids = get_trip_ids(service_id, route_id, direction)
            route_variations = how_many_variants(trip_ids)
            if route_variations:
                beginning_subroutes += len(route_variations)
                if len(route_variations) == 1:
                    keep_variations[subroute]= route_variations[0]
                else:
    
                    route_variations_dict = {}
                    for variation in route_variations:
                        stops = get_list_of_stops(variation)

                        new_stops = []
                        
                        for stop in stops:
                        
                            #print(stop)
                            new_stop = reverse_stop_formatter(stop)
                            new_stops.append(new_stop)
                            
                        route_variations_dict[variation] = new_stops
                    
                    new_list = []
                    
                    for key3, value3 in route_variations_dict.items():
                        new_list.append(str(value3))
                    
            
                    duplicates_removed_list = list(set(new_list))
                    
                            
                    unique_list = duplicates_removed_list
                    
                    
                    #count += len(duplicates_removed_list) - len(unique_list)
                    #time to get the unique list back into lists rather than strings:
                    unique_list_list = []
                    for key3, value3 in route_variations_dict.items():
                        if str(value3) in unique_list and value3 not in unique_list_list:
                            unique_list_list.append(value3)
                
                        
                    if len(unique_list_list) > 1:
                        
                        if len(unique_list_list) == 2:
                            #print("2", route, direction)
                            if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))== 0 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) == 0:
                                unique_list_list.remove(min(unique_list_list))
                            elif len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) <= 2:
                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[1])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[0])
                                
                                exceptions.append([route, direction, key3, abnormality])
                                
                                unique_list_list.remove(min(unique_list_list))
                                
                        if len(unique_list_list) > 2:
                            removal_list = []
                            for i in range(0, len(unique_list_list)-1):
                                for j in range(i, i+1):
                                    if len(np.setdiff1d(unique_list_list[i], unique_list_list[i+1]))<= 2 or len(np.setdiff1d(unique_list_list[i+1],unique_list_list[i])) <= 2:
                                        removal_list.append(min(unique_list_list[i], unique_list_list[i+1]))
                        
                        
                        
                        if len(unique_list_list) == 3:
                                #print("3", route, direction)
                                removal_list = []
                            #if len(np.setdiff1d(unique_list_list[0], unique_list_list[1]))== 0 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) == 0:
                                        #removal_list.append(min(unique_list_list[0], unique_list_list[1]))
                            #elif len(np.setdiff1d(unique_list_list[0], unique_list_list[2]))== 0 or len(np.setdiff1d(unique_list_list[2],unique_list_list[0])) == 0:
                                        #removal_list.append(min(unique_list_list[0], unique_list_list[2]))
                            #elif len(np.setdiff1d(unique_list_list[1], unique_list_list[2]))== 0 or len(np.setdiff1d(unique_list_list[2],unique_list_list[1])) == 0:
                                        #removal_list.append(min(unique_list_list[1], unique_list_list[2]))
                                
                                
                            #if len(unique_list_list) ==3:
                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) <= 2:
                                    #print("0,1")
                                    if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2:
                                        abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[1])
                                    
                                    else:
                                        abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[0])
                                        
                                    removal_list.append(min(unique_list_list[1], unique_list_list[0]))
                                    exceptions.append([route, direction, key3, abnormality])

                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[2]))<= 2 or len(np.setdiff1d(unique_list_list[2],unique_list_list[0])) <= 2:
                                    #print("0,2")
                                    if len(np.setdiff1d(unique_list_list[0],unique_list_list[2]))<= 2:
                                        abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[2])
                                    else:
                                        abnormality = np.setdiff1d(unique_list_list[2],unique_list_list[0])
                                        
                                    removal_list.append(min(unique_list_list[2], unique_list_list[0]))
                                    exceptions.append([route, direction, key3, abnormality])

                                if len(np.setdiff1d(unique_list_list[1],unique_list_list[2]))<= 2 or len(np.setdiff1d(unique_list_list[2],unique_list_list[1])) <= 2:
                                    #print("1,2")
                                    if len(np.setdiff1d(unique_list_list[1],unique_list_list[2]))<= 2:
                                        abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[2])
                                    else:
                                        abnormality = np.setdiff1d(unique_list_list[2],unique_list_list[1])
                                  
                                        
                                    removal_list.append(min(unique_list_list[2], unique_list_list[1]))
                                    exceptions.append([route, direction, key3, abnormality])
                        
                        
                        
                            #if len(unique_list_list) == 4:
                                #removal_list = []
                                #if len(np.setdiff1d(unique_list_list[0], unique_list_list[1]))== 0 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) == 0:
                                            #removal_list.append(min(unique_list_list[0], unique_list_list[1]))
                                #elif len(np.setdiff1d(unique_list_list[0], unique_list_list[2]))== 0 or len(np.setdiff1d(unique_list_list[2],unique_list_list[0])) == 0:
                                            #removal_list.append(min(unique_list_list[0], unique_list_list[2]))
                                #elif len(np.setdiff1d(unique_list_list[0], unique_list_list[3]))== 0 or len(np.setdiff1d(unique_list_list[3],unique_list_list[0])) == 0:
                                            #removal_list.append(min(unique_list_list[0], unique_list_list[3]))
                                #elif len(np.setdiff1d(unique_list_list[1], unique_list_list[2]))== 0 or len(np.setdiff1d(unique_list_list[2],unique_list_list[1])) == 0:
                                            #removal_list.append(min(unique_list_list[1], unique_list_list[2]))
                                #elif len(np.setdiff1d(unique_list_list[1], unique_list_list[3]))== 0 or len(np.setdiff1d(unique_list_list[3],unique_list_list[1])) == 0:
                                            #removal_list.append(min(unique_list_list[1], unique_list_list[3]))
                                #elif len(np.setdiff1d(unique_list_list[2], unique_list_list[3]))== 0 or len(np.setdiff1d(unique_list_list[3],unique_list_list[2])) == 0:
                                            #removal_list.append(min(unique_list_list[2], unique_list_list[3]))
                                
                                
                        if len(unique_list_list) == 4:
                            #print("4", route, direction)
                            removal_list = []
                            if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2 or len(np.setdiff1d(unique_list_list[1],unique_list_list[0])) <= 2:
                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[1]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[1])

                                else:
                                    abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[0])

                                removal_list.append(min(unique_list_list[1], unique_list_list[0]))
                                exceptions.append([route, direction, key3, abnormality])

                            if len(np.setdiff1d(unique_list_list[0],unique_list_list[2]))<= 2 or len(np.setdiff1d(unique_list_list[2],unique_list_list[0])) <= 2:
                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[2]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[2])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[2],unique_list_list[0])

                                removal_list.append(min(unique_list_list[0], unique_list_list[2]))
                                exceptions.append([route, direction, key3, abnormality])

                            if len(np.setdiff1d(unique_list_list[0],unique_list_list[3]))<= 2 or len(np.setdiff1d(unique_list_list[3],unique_list_list[0])) <= 2:
                                if len(np.setdiff1d(unique_list_list[0],unique_list_list[3]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[0],unique_list_list[3])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[3],unique_list_list[0])

                                removal_list.append(min(unique_list_list[0], unique_list_list[3]))
                                exceptions.append([route, direction, key3, abnormality])

                            if len(np.setdiff1d(unique_list_list[1],unique_list_list[2]))<= 2 or len(np.setdiff1d(unique_list_list[2],unique_list_list[1])) <= 2:
                                if len(np.setdiff1d(unique_list_list[1],unique_list_list[2]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[2])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[2],unique_list_list[1])

                                removal_list.append(min(unique_list_list[1], unique_list_list[2]))
                                exceptions.append([route, direction, key3, abnormality])

                            if len(np.setdiff1d(unique_list_list[1],unique_list_list[3]))<= 2 or len(np.setdiff1d(unique_list_list[3],unique_list_list[1])) <= 2:
                                if len(np.setdiff1d(unique_list_list[1],unique_list_list[3]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[1],unique_list_list[3])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[3],unique_list_list[1])

                                removal_list.append(min(unique_list_list[1], unique_list_list[3]))
                                exceptions.append([route, direction, key3, abnormality])

                            if len(np.setdiff1d(unique_list_list[2],unique_list_list[3]))<= 2 or len(np.setdiff1d(unique_list_list[3],unique_list_list[2])) <= 2:
                                if len(np.setdiff1d(unique_list_list[2],unique_list_list[3]))<= 2:
                                    abnormality = np.setdiff1d(unique_list_list[2],unique_list_list[3])
                                else:
                                    abnormality = np.setdiff1d(unique_list_list[3],unique_list_list[2])

                                removal_list.append(min(unique_list_list[2], unique_list_list[3]))
                                exceptions.append([route, direction, key3, abnormality])


                        final_list = [x for x in unique_list_list if x not in removal_list]
                    
                    else: 
                        final_list = unique_list_list
                    
                    #print()
                    #count += len(new_list) - len(final_list)
                    #count1 += len(unique_list_list) - len(final_list)
                    
                    problematic_routes += len(final_list)
                    
                    if len(final_list) > 1:
                        print(route, direction)
                        count +=1
                        #print ("Length of final list: ", len(final_list))
                        for i in final_list:
                            print(i)
                        
                        
                    #I need to make two additional changes...
                        #have the code recognise when there is just one stop different
                        #have the code recognise when subroutes are not sequential
                    if len(unique_list_list) == 1:
                        pass # I need to get their trip_ids and add them to keep_variations
                    else:
                        pass
            else:
                not_operating.append(route)
print(exceptions)
print("Overall: ", count)
#print("last-stage", count1)
#print("result: ", problematic_routes)

#print("Not operating", not_operating)
#print("Keeping", keep_variations)

            

41X 1
['4953', '768', '769', '770', '771', '772', '773', '774', '775', '776', '777', '906', '909', '790', '7585', '7587', '407', '2498', '2499', '2500', '2501', '7623', '3676', '5073', '3679', '5075', '5076', '3682', '3864', '3865', '4910', '4911', '4912', '4913', '4914', '4915', '7654']
['4953', '768', '769', '770', '771', '772', '773', '774', '775', '776', '777', '906', '909', '790', '7585', '7587', '407', '2498', '2499', '2500', '2501', '7623', '5030', '5031', '6033', '3089', '3090', '3706', '3707', '3708', '3709', '3710', '4925', '3679', '5075', '5076', '3682', '3864', '3865', '4910', '4911', '4912', '4913', '4914', '4915', '4957']
41X 2
['7654', '4843', '4905', '4906', '4907', '4908', '4909', '3884', '3686', '5077', '3694', '3695', '7115', '6310', '6311', '4495', '2809', '2810', '842', '845', '846', '847', '848', '2795', '756', '757', '758', '759', '760', '762', '763', '4953']
['3088', '3704', '3705', '733', '6015', '5029', '5028', '7115', '6310', '6311', '4495', '2809', '2810', '

66X 2
['7313', '6164', '7712', '6159', '6160', '6161', '3991', '3998', '3999', '4000', '4001', '7193', '7049', '2236', '2239', '2241', '2242', '7435', '2722', '4413', '1476', '7453', '1478', '1479', '315', '4495', '2809', '2810', '749', '750', '751', '752', '753', '754', '755', '756', '757', '758', '759', '760', '761', '762', '763', '4952']
['3994', '3995', '3996', '3998', '3999', '4000', '4001', '7193', '7049', '2236', '2239', '2241', '2242', '7435', '2722', '4413', '1476', '7453', '1478', '1479', '315', '4495', '2809', '2810', '749', '750', '751', '752', '753', '754', '755', '756', '757', '758', '759', '760', '761', '762', '763', '4952']
['3980', '6089', '3981', '3982', '7704', '3983', '3985', '3986', '3987', '3989', '3990', '3991', '3998', '3999', '4000', '4001', '7193', '7049', '2236', '2239', '2241', '2242', '7435', '2722', '4413', '1476', '7453', '1478', '1479', '315', '4495', '2809', '2810', '749', '750', '751', '752', '753', '754', '755', '756', '757', '758', '759', '760', '761

27B 2
['1176', '1177', '1178', '1179', '1238', '1239', '247', '1240', '1241', '1242', '1277', '1219', '1220', '1221', '664', '665', '666', '667', '668', '614', '615', '616', '617', '618', '619', '675', '303']
['324', '1634', '1635', '1636', '1637', '1638', '992', '7044', '993', '5060', '5061', '1231', '1232', '1233', '1234', '7358', '1235', '1251', '7130', '1237', '1238', '1239', '247', '1240', '1241', '1242', '1277', '1219', '1220', '1221', '664', '665', '666', '667', '668', '614', '615', '616', '617', '618', '619', '675', '303']
['7044', '993', '5060', '5061', '1231', '1232', '1233', '1234', '7358', '1235', '1251', '253', '7130', '1237', '1238', '1239', '247', '1240', '1241', '1242', '1277', '1219', '1220', '1221', '664', '665', '666', '667', '668', '614', '615', '616', '617', '618', '619', '675', '303']
27 1
['4595', '4596', '4563', '1218', '1258', '1259', '1260', '6130', '1262', '4692', '1245', '4693', '1254', '1255', '1256', '4694', '1249', '1247', '1248', '1273', '1274', '1275', 

In [137]:
print(len(exceptions))

66


In [139]:
print(beginning_subroutes)

722


In [140]:
len(keep_variations)

79

In [51]:
len(not_operating)

81

#### what do we need to do now:
    - figure out which variations are being kept (are they actually different in terms of stops?)
    - create a new ordered stop list for the front end with names based on GTFS data
    - create a list of model exceptions 
        - allowing models to handle stops on route that were not present in 2018
        - extending time for additions to route
        - ignore city centre diversions - just assume time is similar
    

In [41]:
a = ['4525', '4526', '1700', '1701', '1702', '16', '73', '74', '4371', '832', '833', '77', '78', '79', '80', '82', '83', '84', '51', '52', '263', '271', '1278', '1282', '4456', '1284', '1285', '1286', '1288', '1289', '1360', '1362', '1363', '1364', '1365', '1366', '1367', '7455', '7438', '1410', '1411', '1412', '1413', '1414', '1415', '1417', '1418', '5149', '1419', '6094']

b = str(['4525', '4526', '1700', '1701', '1702', '16', '73', '74', '4371', '832', '833', '77', '78', '79', '80', '82', '83', '84', '51', '52', '263'])

In [42]:
str(a) in str(b)

False

In [48]:
str(b).strip(']') in str(a)

True

In [ ]:
['6235', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '2167', '3383', '3384', '7230', '4552', '4554', '7229']
['6235', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270']
['6235', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '7229']
['6235', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '4545', '4552', '4554', '7229']
['324', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '4545', '4552', '4554', '7229']
['324', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '7229']
['324', '327', '6016', '6017', '7192', '4683', '107', '6125', '7092', '110', '112', '113', '114', '115', '37', '38', '39', '116', '117', '118', '119', '44', 0, '45', '46', '47', '48', '49', '51', '52', '261', '270', '335', '4521', '1935', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '4379', '1963', '4549', '1964', '4674', '4663', '4664', '2160', '2161', '4665', '4666', '4667', '4668', '4677', '2136', '2137', '7201', '2138', '2139', '2140', '2141', '2142', '2143', '2145', '4670', '2165', '2166', '2167', '3383', '3384', '7230', '4552', '4554', '7229']

so it seems that in the mornings the 13 stops at the 324... but after a certain time it stops at 6235 (a stop around the corner instead - presumably this is to do with if the bus is leaving the garage or has come back from a journey)
the second row should be deleted....but why didn't it get removed by my subset scanner...?
also.... there are three possible endings for the 13
and it's basically an insertion of 3 or 5 extra stops...
so I think we should use the bottom list....or the top list.
I need to make a decision about how to manage that...if it's a easy cut off time I could have different stops listed at different times of day...except that won't work because... I don't know the users time when they select a route.

In [ ]:
['4863', '4864', '4865', '4577', '4578', '1100', '100', '1111', '2323', '2324', '2325', '2326', '5150', '2327', '2328', '2329', '2330', '2331', '1397', '1398', '1399', '5148', '1401', '1402', '1403', '1404', '1405', '4856', '4857', '307', '308', '309', '2383', '2003', '1358', '4494']
['4863', '4864', '4865', '4577', '4578', '1100', '100', '1111', '2323', '2324', '2325', '2326', '5150', '2327', '2328', '2329', '2330', '2331', '1397', '1398', '1399', '5148', '1401', '1402', '1403', '1404', '1405', '4856', '4857', '307', '308', '309', '7412', '7413', '2383', '2003', '1358', '4494']

In [ ]:
                                                                                          ['7333', '783', '784', '785', '786', '793', '7586', '7587', '7588', '328', '1443', '1444', '1445', '1647', '1648', '1649', '1911', '1913', '1914', '1805', '1806', '1660', '1661', '1662', '1664', '1665', '1666', '1807', '7167', '1808', '1813', '1814', '1852', '1853', '1854', '1844', '6172', '4493', '4362', '7025', '4464', '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '1899', '1887', '1888', '1889', '1890', '7240', '1891', '7156', '7020', '7029', '7038', '7011', '2171', '7160', '7700', '7047', '7161', '7162']
['767', '768', '769', '770', '771', '772', '773', '774', '775', '776', '777', '779', '780', '781', '782', '783', '784', '785', '786', '793', '7586', '7587', '7588', '328', '1443', '1444', '1445', '1647', '1648', '1649', '1911', '1913', '1914', '1805', '1806', '1660', '1661', '1662', '1664', '1665', '1666', '1807', '7167', '1808', '1813', '1814', '1852', '1853', '1854', '1844', '6172', '4493', '4362', '7025', '4464', '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '1899', '1887', '1888', '1889', '1890', '7240', '1891', '7156', '7020', '7029', '7038', '7011', '2171', '7160', '7700', '7047', '7161', '7162']

In [ ]:
['807', '808', '809', '810', '811', '812', '813', '814', '81', '817', '818', '819', '264', '6059', '334', '406', '747', '842', '845', '846', '847', '848', '2795', '756', '757', '758', '759', '760', '761', '762', '763', '2007', '2008', '2009', '2010', '435', '7353', '4571', '2013', '2014', '4636', '2016', '2015', '2017', '2018', '2019', '2020', '2021', '2022', '4565', '4566', '4567', '2031', '2032', '2033', '2034', '2035', '2036', '2039']
['807', '808', '809', '810', '811', '812', '813', '814', '81', '817', '818', '819', '264', '6059', '334', '406', '747', '842', '845', '846', '847', '848', '2795', '756', '757', '758', '759', '760', '761', '762', '763', '767']
['807', '808', '809', '810', '811', '812', '813', '814', '81', '817', '818', '819', '264', '6059', '334']

In [ ]:
122 1
Shortest:  ['4525', '4526', '1700', '1701', '1702', '16', '73', '74', '4371', '832', '833', '77', '78', '79', '80', '82', '83', '84', '51', '52', '263']
Unique list: [['4525', '4526', '1700', '1701', '1702', '16', '73', '74', '4371', '832', '833', '77', '78', '79', '80', '82', '83', '84', '51', '52', '263', '271', '1278', '1282', '4456', '1284', '1285', '1286', '1288', '1289', '1360', '1362', '1363', '1364', '1365', '1366', '1367', '7455', '7438', '1410', '1411', '1412', '1413', '1414', '1415', '1417', '1418', '5149', '1419', '6094'], 
              ['4525', '4526', '1700', '1701', '1702', '16', '73', '74', '4371', '832', '833', '77', '78', '79', '80', '82', '83', '84', '51', '52', '263']]